#Cargando todas las bases que necesitamos

##Clientes 2024 (Base de Julio antiguas Curvas de Liquidación)

In [1]:
def running_in_colab():
    try:
        import google.colab  # noqa: F401
        return True
    except ImportError:
        return False


if running_in_colab():
    !pip install --quiet gspread google-auth pandas

In [2]:
"""
Lee la ÚNICA hoja del archivo de Google Sheets (o Excel en Drive) cuyo link es:

https://docs.google.com/spreadsheets/d/12gi1482pwvg01HWt83Fsw5N9L25vV3FHO1pHwIFkds0/edit?gid=0#gid=0

- En Colab:
    Usa el secreto MI_JSON desde google.colab.userdata (NO desde os.environ).
    Opcionalmente puede leer CORREO_COLAB solo para mostrarlo en pantalla.

- En GitHub / local:
    Intenta leer MI_JSON desde la variable de entorno MI_JSON.
    Si no existe, intenta usar un archivo local 'service_account.json'.

Devuelve un DataFrame de pandas llamado df_curvas (única hoja, índice 0).
"""

import io
import json
import os

import pandas as pd

from google.oauth2.service_account import Credentials
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError


# ============================================================
# 1. Parámetros del archivo
# ============================================================

SPREADSHEET_URL = (
    "https://docs.google.com/spreadsheets/d/"
    "12gi1482pwvg01HWt83Fsw5N9L25vV3FHO1pHwIFkds0/edit?gid=0#gid=0"
)


def extract_file_id_from_url(url: str) -> str:
    """
    Extrae el fileId de una URL tipo:
    https://docs.google.com/spreadsheets/d/<ID>/edit?...
    """
    try:
        part = url.split("/d/")[1]
        file_id = part.split("/")[0]
        return file_id
    except IndexError:
        raise ValueError(
            f"No se pudo extraer el fileId desde la URL: {url}. "
            "Revisa que tenga el formato .../d/<ID>/..."
        )


# ============================================================
# 2. Detectar si estamos en Google Colab
# ============================================================

def running_in_colab() -> bool:
    try:
        import google.colab  # noqa: F401
        return True
    except ImportError:
        return False


# ============================================================
# 3. Obtener credenciales
# ============================================================

def get_credentials():
    """
    Devuelve un objeto Credentials para usar con las APIs de Google.

    - En Colab:
        Lee MI_JSON desde google.colab.userdata (NUNCA desde os.environ).
    - Fuera de Colab:
        Intenta leer MI_JSON desde os.environ["MI_JSON"].
        Si no existe, intenta leer 'service_account.json' local.
    """
    scopes = [
        "https://www.googleapis.com/auth/drive",
        "https://www.googleapis.com/auth/spreadsheets.readonly",
    ]

    if running_in_colab():
        from google.colab import userdata

        mi_json = userdata.get("MI_JSON")
        if mi_json is None:
            raise RuntimeError(
                "En Colab, el secreto 'MI_JSON' no está configurado en userdata.\n"
                "Ve a: Entorno de ejecución -> Configurar secretos, y crea 'MI_JSON'."
            )

        service_info = json.loads(mi_json)

        # (Opcional) Ver CORREO_COLAB solo para que tú verifiques permisos
        correo_colab = userdata.get("CORREO_COLAB")
        if correo_colab:
            print("CORREO_COLAB (para compartir el archivo):", correo_colab)

        creds = Credentials.from_service_account_info(service_info, scopes=scopes)
        return creds

    else:
        # Entorno GitHub / local
        mi_json = os.environ.get("MI_JSON")

        if mi_json is not None:
            service_info = json.loads(mi_json)
            creds = Credentials.from_service_account_info(service_info, scopes=scopes)
            return creds

        cred_path = "service_account.json"
        if os.path.exists(cred_path):
            with open(cred_path, "r", encoding="utf-8") as f:
                service_info = json.load(f)
            creds = Credentials.from_service_account_info(service_info, scopes=scopes)
            return creds

        raise RuntimeError(
            "No se encontraron credenciales para Google.\n"
            "- En GitHub/local: define la variable de entorno MI_JSON con el contenido\n"
            "  del JSON de servicio, o crea 'service_account.json' en el directorio actual."
        )


# ============================================================
# 4. Descargar el archivo de Drive como .xlsx
# ============================================================

def download_file_as_xlsx_bytes(file_id: str, creds: Credentials) -> bytes:
    """
    Descarga un archivo de Google Drive como .xlsx y devuelve el contenido en bytes.

    - Si es una hoja nativa de Google Sheets -> files().export(...).
    - Si es un Excel subido (u otro tipo descargable) -> files().get_media(...).
    """
    drive_service = build("drive", "v3", credentials=creds)

    # 1) Intentar exportar como .xlsx (Google Sheets)
    try:
        request = drive_service.files().export(
            fileId=file_id,
            mimeType="application/vnd.openxmlformats-officedocument.spreadsheetml.sheet",
        )
        fh = io.BytesIO()
        fh.write(request.execute())
        fh.seek(0)
        return fh.read()

    except HttpError as e:
        # Si no es exportable (no es Google Sheet), probamos descargar el archivo original
        if e.resp.status in (400, 404, 500):
            request = drive_service.files().get_media(fileId=file_id)
            fh = io.BytesIO()
            fh.write(request.execute())
            fh.seek(0)
            return fh.read()
        else:
            raise


# ============================================================
# 5. Cargar la ÚNICA hoja a DataFrame
# ============================================================

def load_curvas_dataframe(spreadsheet_url: str = SPREADSHEET_URL) -> pd.DataFrame:
    """
    Descarga el archivo desde Drive como .xlsx y carga la ÚNICA hoja (sheet 0)
    en un DataFrame de pandas.
    """
    creds = get_credentials()
    file_id = extract_file_id_from_url(spreadsheet_url)

    xlsx_bytes = download_file_as_xlsx_bytes(file_id, creds)

    # Leemos el Excel desde bytes con pandas (sheet 0 = única hoja)
    with io.BytesIO(xlsx_bytes) as bio:
        df = pd.read_excel(bio, sheet_name=0)

    return df


# ============================================================
# 6. Punto de entrada
# ============================================================

if __name__ == "__main__":
    df_curvas = load_curvas_dataframe()

    print("DataFrame 'df_curvas' cargado correctamente.")
    print("Shape:", df_curvas.shape)
    print("Columnas:", list(df_curvas.columns))
    print("\nPrimeras filas:")
    print(df_curvas.head())

CORREO_COLAB (para compartir el archivo): colabservice@colabaccess-468021.iam.gserviceaccount.com
DataFrame 'df_curvas' cargado correctamente.
Shape: (88814, 19)
Columnas: ['Referencia', 'status', 'Status', 'start_date', 'fecha_de_baja', 'Liquidación', 'fecha_de_graduacion', 'Deuda Bravo', 'Deuda Liquidada', 'Deuda Baja', 'Mes Baja', 'Mes Liquidado', 'Mes Graduado', 'Mes Actualizado', '% Volumetría', 'Liquidado Volumetría', 'Liquidado Volumetría.1', 'Fecha Primera Act', 'Fecha Última Act']

Primeras filas:
   Referencia           status Status start_date fecha_de_baja Liquidación  \
0  3192497430  Baja Solicitada   Baja 2020-09-30    2020-10-31         NaT   
1  3105766139     Baja Saldo 0   Baja 2020-10-26    2020-12-31         NaT   
2  3213068543  Baja Solicitada   Baja 2020-02-27    2020-10-31         NaT   
3  3166990462         Baja 2xC   Baja 2020-01-04    2020-04-30         NaT   
4  3196337371         Baja 2xC   Baja 2020-01-15    2020-04-30         NaT   

  fecha_de_graduaci

In [3]:
df_curvas

,Referencia,status,Status,start_date,fecha_de_baja,Liquidación,fecha_de_graduacion,Deuda Bravo,Deuda Liquidada,Deuda Baja,Mes Baja,Mes Liquidado,Mes Graduado,Mes Actualizado,% Volumetría,Liquidado Volumetría,Liquidado Volumetría.1,Fecha Primera Act,Fecha Última Act
0,3192497430,Baja Solicitada,Baja,2020-09-30,2020-10-31,NaT,NaT,6620900,NaN,6620900.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT
1,3105766139,Baja Saldo 0,Baja,2020-10-26,2020-12-31,NaT,NaT,26121602,NaN,26121602.0,2.0,NaN,NaN,NaN,NaN,52881902.0,NaN,NaT,NaT
2,3213068543,Baja Solicitada,Baja,2020-02-27,2020-10-31,NaT,NaT,6370050,NaN,6370050.0,8.0,NaN,NaN,NaN,NaN,3627750.0,NaN,NaT,NaT
3,3166990462,Baja 2xC,Baja,2020-01-04,2020-04-30,NaT,NaT,18242400,NaN,18242400.0,3.0,NaN,NaN,NaN,NaN,4830100.0,NaN,NaT,NaT
4,3196337371,Baja 2xC,Baja,2020-01-15,2020-04-30,NaT,NaT,6556150,NaN,6556150.0,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
88809,3112976609,Activo,Activo,2024-12-31,NaT,NaT,NaT,13299902,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11322150.0,NaN,NaT,NaT
88810,3002816928,Crédito Parcial,Crédito,2024-12-31,2025-07-31,2025-07-16,NaT,5045250,1134000.0,3911250.0,7.0,6.0,NaN,NaN,NaN,1921500.0,1134000.0,NaT,NaT
88811,3135810159,Baja 2xC,Baja,2024-12-31,2025-03-31,NaT,NaT,34884150,NaN,34884150.0,3.0,NaN,NaN,NaN,NaN,34884150.0,NaN,NaT,NaT
88812,3112787565,Activo,Activo,2024-12-31,NaT,2025-06-27,NaT,21672000,17932950.0,NaN,NaN,5.0,NaN,NaN,NaN,NaN,NaN,NaT,NaT


In [4]:
columnas_eliminar = ['Status']
df_curvas = df_curvas.drop(columnas_eliminar, axis=1)

In [5]:
import pandas as pd
import unicodedata
import numpy as np


# ============================================================
# 1. Normalizar nombres de columnas (MAYÚSCULAS + SIN TILDES)
# ============================================================

def normalize_column(colname: str) -> str:
    # Convertir a mayúsculas
    colname = colname.upper()

    # Remover tildes
    colname = (
        unicodedata.normalize("NFKD", colname)
        .encode("ASCII", "ignore")
        .decode("ASCII")
    )

    # Reemplazar espacios por "_"
    colname = colname.replace(" ", "_")

    return colname


df = df_curvas.copy()
df.columns = [normalize_column(c) for c in df.columns]


# ============================================================
# 2. REFERENCIA siempre como OBJECT
# ============================================================

if "REFERENCIA" in df.columns:
    df["REFERENCIA"] = df["REFERENCIA"].astype("object")


# ============================================================
# 3. Detección automática de columnas FECHA → datetime
# ============================================================

def is_date_series(series: pd.Series) -> bool:
    try:
        pd.to_datetime(series.dropna().astype(str), errors="raise")
        return True
    except:
        return False


for col in df.columns:
    if col == "REFERENCIA":
        continue
    if df[col].dtype == "object":
        if is_date_series(df[col]):
            df[col] = pd.to_datetime(df[col], errors="coerce")


# ============================================================
# 4. Columnas numéricas → float
# ============================================================

def can_be_numeric(series: pd.Series) -> bool:
    s = pd.to_numeric(series.astype(str).str.replace(",", ""), errors="coerce")
    # Si más del 80% de valores son numéricos, se considera numérica
    return s.notna().mean() > 0.8


for col in df.columns:
    if col == "REFERENCIA":
        continue

    # Si ya es datetime, no lo tocamos
    if pd.api.types.is_datetime64_any_dtype(df[col]):
        continue

    if df[col].dtype == "object":
        if can_be_numeric(df[col]):
            df[col] = (
                pd.to_numeric(df[col].astype(str).str.replace(",", ""), errors="coerce")
            )
        else:
            df[col] = df[col].astype("object")

    # Si es int → convertir a float (para evitar errores futuros)
    elif pd.api.types.is_integer_dtype(df[col]):
        df[col] = df[col].astype(float)


# ============================================================
# Resultado final
# ============================================================

print("Transformaciones completadas.")
print(df.info())
df.head()

/tmp/ipython-input-928463830.py:45: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  pd.to_datetime(series.dropna().astype(str), errors="raise")


Transformaciones completadas.
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 88814 entries, 0 to 88813
Data columns (total 18 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   REFERENCIA              88814 non-null  object        
 1   STATUS                  88814 non-null  object        
 2   START_DATE              88814 non-null  datetime64[ns]
 3   FECHA_DE_BAJA           76202 non-null  datetime64[ns]
 4   LIQUIDACION             23870 non-null  datetime64[ns]
 5   FECHA_DE_GRADUACION     10128 non-null  datetime64[ns]
 6   DEUDA_BRAVO             88814 non-null  float64       
 7   DEUDA_LIQUIDADA         23870 non-null  float64       
 8   DEUDA_BAJA              73177 non-null  float64       
 9   MES_BAJA                71855 non-null  float64       
 10  MES_LIQUIDADO           23521 non-null  float64       
 11  MES_GRADUADO            10127 non-null  float64       
 12  MES_ACTUALIZADO 

,REFERENCIA,STATUS,START_DATE,FECHA_DE_BAJA,LIQUIDACION,FECHA_DE_GRADUACION,DEUDA_BRAVO,DEUDA_LIQUIDADA,DEUDA_BAJA,MES_BAJA,MES_LIQUIDADO,MES_GRADUADO,MES_ACTUALIZADO,%_VOLUMETRIA,LIQUIDADO_VOLUMETRIA,LIQUIDADO_VOLUMETRIA.1,FECHA_PRIMERA_ACT,FECHA_ULTIMA_ACT
0,3192497430,Baja Solicitada,2020-09-30,2020-10-31,NaT,NaT,6620900.0,NaN,6620900.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT
1,3105766139,Baja Saldo 0,2020-10-26,2020-12-31,NaT,NaT,26121602.0,NaN,26121602.0,2.0,NaN,NaN,NaN,NaN,52881902.0,NaN,NaT,NaT
2,3213068543,Baja Solicitada,2020-02-27,2020-10-31,NaT,NaT,6370050.0,NaN,6370050.0,8.0,NaN,NaN,NaN,NaN,3627750.0,NaN,NaT,NaT
3,3166990462,Baja 2xC,2020-01-04,2020-04-30,NaT,NaT,18242400.0,NaN,18242400.0,3.0,NaN,NaN,NaN,NaN,4830100.0,NaN,NaT,NaT
4,3196337371,Baja 2xC,2020-01-15,2020-04-30,NaT,NaT,6556150.0,NaN,6556150.0,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT


In [6]:
df_2024 = df

In [7]:
df_2024

,REFERENCIA,STATUS,START_DATE,FECHA_DE_BAJA,LIQUIDACION,FECHA_DE_GRADUACION,DEUDA_BRAVO,DEUDA_LIQUIDADA,DEUDA_BAJA,MES_BAJA,MES_LIQUIDADO,MES_GRADUADO,MES_ACTUALIZADO,%_VOLUMETRIA,LIQUIDADO_VOLUMETRIA,LIQUIDADO_VOLUMETRIA.1,FECHA_PRIMERA_ACT,FECHA_ULTIMA_ACT
0,3192497430,Baja Solicitada,2020-09-30,2020-10-31,NaT,NaT,6620900.0,NaN,6620900.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT
1,3105766139,Baja Saldo 0,2020-10-26,2020-12-31,NaT,NaT,26121602.0,NaN,26121602.0,2.0,NaN,NaN,NaN,NaN,52881902.0,NaN,NaT,NaT
2,3213068543,Baja Solicitada,2020-02-27,2020-10-31,NaT,NaT,6370050.0,NaN,6370050.0,8.0,NaN,NaN,NaN,NaN,3627750.0,NaN,NaT,NaT
3,3166990462,Baja 2xC,2020-01-04,2020-04-30,NaT,NaT,18242400.0,NaN,18242400.0,3.0,NaN,NaN,NaN,NaN,4830100.0,NaN,NaT,NaT
4,3196337371,Baja 2xC,2020-01-15,2020-04-30,NaT,NaT,6556150.0,NaN,6556150.0,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
88809,3112976609,Activo,2024-12-31,NaT,NaT,NaT,13299902.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11322150.0,NaN,NaT,NaT
88810,3002816928,Crédito Parcial,2024-12-31,2025-07-31,2025-07-16,NaT,5045250.0,1134000.0,3911250.0,7.0,6.0,NaN,NaN,NaN,1921500.0,1134000.0,NaT,NaT
88811,3135810159,Baja 2xC,2024-12-31,2025-03-31,NaT,NaT,34884150.0,NaN,34884150.0,3.0,NaN,NaN,NaN,NaN,34884150.0,NaN,NaT,NaT
88812,3112787565,Activo,2024-12-31,NaT,2025-06-27,NaT,21672000.0,17932950.0,NaN,NaN,5.0,NaN,NaN,NaN,NaN,NaN,NaT,NaT


##Bases de Metabase

In [8]:
import os
import requests
import pandas as pd
from io import StringIO
from requests.exceptions import JSONDecodeError

BASE_URL = "https://metabase.resuelve.io"
CARD_ID = 10312  # resumen-clientes-deudas-clientes-baja


def running_in_colab():
    try:
        import google.colab  # noqa
        return True
    except ImportError:
        return False


def get_mb_credentials():
    user = None
    password = None

    if running_in_colab():
        from google.colab import userdata
        user = userdata.get("MB_USER")
        password = userdata.get("MB_PASS")

    if not user:
        user = os.environ.get("MB_USER")
    if not password:
        password = os.environ.get("MB_PASS")

    if not user or not password:
        raise ValueError("No encontré credenciales de Metabase.")

    return user, password


def create_metabase_session():
    user, password = get_mb_credentials()

    resp = requests.post(
        f"{BASE_URL}/api/session",
        json={"username": user, "password": password},
        timeout=60,
    )
    resp.raise_for_status()

    session_id = resp.json()["id"]
    headers = {"X-Metabase-Session": session_id}
    return headers


def get_card_full(card_id: int) -> pd.DataFrame:
    headers = create_metabase_session()

    try:
        resp = requests.post(
            f"{BASE_URL}/api/card/{card_id}/query/json",
            headers=headers,
            json={},
            timeout=600,
        )
        resp.raise_for_status()
        rows = resp.json()
        return pd.DataFrame(rows)

    except (JSONDecodeError, ValueError):
        resp = requests.post(
            f"{BASE_URL}/api/card/{card_id}/query/csv",
            headers=headers,
            json={},
            timeout=600,
        )
        resp.raise_for_status()
        return pd.read_csv(StringIO(resp.text))


df_10312 = get_card_full(CARD_ID)
print(df_10312.shape)
df_10312.head()

(69670, 13)


,id_berex,graduation_date,debt_id,max,id,drop_date,financial_entity_name,inserted_at,liquidated_with_credit,db,paid_off_date,payment_to_bank,bank_reference
0,432204,None,1442113,2024-12-03T16:30:11.602809,1442113,2025-03-25,Banco Popular,2024-02-14T19:09:04,False,42078.5,None,12500.00,3003925128
1,193957,None,645217,2022-01-07T11:08:55,645217,None,Scotiabank Colpatria,2021-01-12T20:29:44,True,16555.0,2021-12-24,8000.00,46622
2,293068,2023-03-16,950756,2023-06-21T00:33:50.591088,950756,None,Colsubsidio,2022-06-06T17:30:23,False,4983.0,2023-03-16,2420.00,3014445964
3,395384,2024-07-22,1307194,2024-07-24T14:25:13.819911,1307194,None,Rappipay,2023-09-08T17:16:08,True,28622.0,2024-07-22,19238.13,1070623204
4,266503,2022-08-17,868155,2022-10-05T17:29:03,868155,None,Davivienda,2022-01-28T19:31:43,True,101346.0,2022-08-17,50000.00,3193745298


In [9]:
df_10312

,id_berex,graduation_date,debt_id,max,id,drop_date,financial_entity_name,inserted_at,liquidated_with_credit,db,paid_off_date,payment_to_bank,bank_reference
0,432204,None,1442113,2024-12-03T16:30:11.602809,1442113,2025-03-25,Banco Popular,2024-02-14T19:09:04,False,42078.5,None,12500.0000,3003925128
1,193957,None,645217,2022-01-07T11:08:55,645217,None,Scotiabank Colpatria,2021-01-12T20:29:44,True,16555.0,2021-12-24,8000.0000,46622
2,293068,2023-03-16,950756,2023-06-21T00:33:50.591088,950756,None,Colsubsidio,2022-06-06T17:30:23,False,4983.0,2023-03-16,2420.0000,3014445964
3,395384,2024-07-22,1307194,2024-07-24T14:25:13.819911,1307194,None,Rappipay,2023-09-08T17:16:08,True,28622.0,2024-07-22,19238.1300,1070623204
4,266503,2022-08-17,868155,2022-10-05T17:29:03,868155,None,Davivienda,2022-01-28T19:31:43,True,101346.0,2022-08-17,50000.0000,3193745298
...,...,...,...,...,...,...,...,...,...,...,...,...,...
69665,526578,2025-12-09,1788454,2025-12-10T14:38:17.874607,1788454,None,BBVA,2025-02-18T19:07:49,True,150087.0,2025-12-09,53800.0000,3102895483
69666,581203,None,1989352,2025-12-10T14:48:21.19567,1989352,None,Cobrando Falabella,2025-09-11T11:47:26,False,69883.0,None,1318.0759,3227677307
69667,576219,None,1970886,2025-12-15T14:07:55.691431,1970886,None,Sistecredito,2025-08-25T16:58:13,False,357.0,2025-12-15,374.5300,3106037019
69668,377169,None,1241912,2023-09-19T22:02:08.914312,1241912,None,Sistecredito,2023-06-16T09:59:40,False,946.0,2023-09-11,1260.0600,3163298069


In [10]:
df_liquidado_2025 = df_10312

In [11]:
df_liquidado_2025["max"] = pd.to_datetime(df_liquidado_2025["max"], format="mixed").dt.date
df_liquidado_2025["inserted_at"] = pd.to_datetime(df_liquidado_2025["inserted_at"], format="mixed").dt.date

In [12]:
df_liquidado_2025["paid_off_date"] = pd.to_datetime(
    df_liquidado_2025["paid_off_date"],
    format="mixed",
    errors="coerce"
)

In [13]:
df_liquidado_2025["paid_off_date"] = df_liquidado_2025["paid_off_date"].fillna(
    df_liquidado_2025["max"]
)

In [14]:
df_liquidado_2025

,id_berex,graduation_date,debt_id,max,id,drop_date,financial_entity_name,inserted_at,liquidated_with_credit,db,paid_off_date,payment_to_bank,bank_reference
0,432204,None,1442113,2024-12-03,1442113,2025-03-25,Banco Popular,2024-02-14,False,42078.5,2024-12-03,12500.0000,3003925128
1,193957,None,645217,2022-01-07,645217,None,Scotiabank Colpatria,2021-01-12,True,16555.0,2021-12-24,8000.0000,46622
2,293068,2023-03-16,950756,2023-06-21,950756,None,Colsubsidio,2022-06-06,False,4983.0,2023-03-16,2420.0000,3014445964
3,395384,2024-07-22,1307194,2024-07-24,1307194,None,Rappipay,2023-09-08,True,28622.0,2024-07-22,19238.1300,1070623204
4,266503,2022-08-17,868155,2022-10-05,868155,None,Davivienda,2022-01-28,True,101346.0,2022-08-17,50000.0000,3193745298
...,...,...,...,...,...,...,...,...,...,...,...,...,...
69665,526578,2025-12-09,1788454,2025-12-10,1788454,None,BBVA,2025-02-18,True,150087.0,2025-12-09,53800.0000,3102895483
69666,581203,None,1989352,2025-12-10,1989352,None,Cobrando Falabella,2025-09-11,False,69883.0,2025-12-10,1318.0759,3227677307
69667,576219,None,1970886,2025-12-15,1970886,None,Sistecredito,2025-08-25,False,357.0,2025-12-15,374.5300,3106037019
69668,377169,None,1241912,2023-09-19,1241912,None,Sistecredito,2023-06-16,False,946.0,2023-09-11,1260.0600,3163298069


In [15]:
import pandas as pd

# Asegurar que las fechas estén en formato datetime
df_liquidado_2025['paid_off_date'] = pd.to_datetime(df_liquidado_2025['paid_off_date'], errors='coerce')
df_liquidado_2025['inserted_at'] = pd.to_datetime(df_liquidado_2025['inserted_at'], errors='coerce')
df_liquidado_2025['max'] = pd.to_datetime(df_liquidado_2025['max'], errors='coerce')

# Crear máscara de filas donde paid_off_date es anterior a inserted_at
mask = df_liquidado_2025['paid_off_date'] < df_liquidado_2025['inserted_at']

# Reemplazar paid_off_date con max donde la condición se cumpla
df_liquidado_2025.loc[mask, 'paid_off_date'] = df_liquidado_2025.loc[mask, 'max']

In [16]:
import os
import requests
import pandas as pd
from io import StringIO
from requests.exceptions import JSONDecodeError

BASE_URL = "https://metabase.resuelve.io"
CARD_ID = 8658  # resumen-clientes-deudas-clientes-baja


def running_in_colab():
    try:
        import google.colab  # noqa
        return True
    except ImportError:
        return False


def get_mb_credentials():
    user = None
    password = None

    if running_in_colab():
        from google.colab import userdata
        user = userdata.get("MB_USER")
        password = userdata.get("MB_PASS")

    if not user:
        user = os.environ.get("MB_USER")
    if not password:
        password = os.environ.get("MB_PASS")

    if not user or not password:
        raise ValueError("No encontré credenciales de Metabase.")

    return user, password


def create_metabase_session():
    user, password = get_mb_credentials()

    resp = requests.post(
        f"{BASE_URL}/api/session",
        json={"username": user, "password": password},
        timeout=60,
    )
    resp.raise_for_status()

    session_id = resp.json()["id"]
    headers = {"X-Metabase-Session": session_id}
    return headers


def get_card_full(card_id: int) -> pd.DataFrame:
    headers = create_metabase_session()

    try:
        resp = requests.post(
            f"{BASE_URL}/api/card/{card_id}/query/json",
            headers=headers,
            json={},
            timeout=600,
        )
        resp.raise_for_status()
        return pd.DataFrame(resp.json())

    except (JSONDecodeError, ValueError):
        resp = requests.post(
            f"{BASE_URL}/api/card/{card_id}/query/csv",
            headers=headers,
            json={},
            timeout=600,
        )
        resp.raise_for_status()
        return pd.read_csv(StringIO(resp.text))


df_8658 = get_card_full(CARD_ID)
print(df_8658.shape)
df_8658.head()

(313949, 9)


,id_berex,id,drop_date,financial_entity_name,status,inserted_at,active_debts,db,bank_reference
0,506867,1716699.0,2025-05-20,Banco de Bogota,new,2024-11-27T15:20:23,2,7933200.0,3024647576
1,506867,1716698.0,2025-05-20,Banco de Bogota,new,2024-11-27T15:20:23,2,2618000.0,3024647576
2,346434,1131486.0,2025-04-29,Davivienda,negotiation,2023-02-13T16:16:11,3,53603000.0,3102856558
3,346434,1131485.0,2025-04-29,Bancolombia,negotiation,2023-02-13T16:16:11,3,35426600.0,3102856558
4,346434,1131484.0,2025-04-29,Éxito,liquidation_structured_payment,2023-02-13T16:16:11,3,15042500.0,3102856558


In [17]:
df_8658

,id_berex,id,drop_date,financial_entity_name,status,inserted_at,active_debts,db,bank_reference
0,506867,1716699.0,2025-05-20,Banco de Bogota,new,2024-11-27T15:20:23,2,7933200.0,3024647576
1,506867,1716698.0,2025-05-20,Banco de Bogota,new,2024-11-27T15:20:23,2,2618000.0,3024647576
2,346434,1131486.0,2025-04-29,Davivienda,negotiation,2023-02-13T16:16:11,3,53603000.0,3102856558
3,346434,1131485.0,2025-04-29,Bancolombia,negotiation,2023-02-13T16:16:11,3,35426600.0,3102856558
4,346434,1131484.0,2025-04-29,Éxito,liquidation_structured_payment,2023-02-13T16:16:11,3,15042500.0,3102856558
...,...,...,...,...,...,...,...,...,...
313944,219613,721433.0,2021-10-31,Davivienda,negotiation,2021-05-19T14:54:46,6,891399.0,76847
313945,219613,721432.0,2021-10-31,Davivienda,negotiation,2021-05-19T14:54:46,6,483940.0,76847
313946,219613,721434.0,2021-10-31,Banco de Bogota,negotiation,2021-05-19T14:54:46,6,5447488.0,76847
313947,219613,721437.0,2021-10-31,Scotiabank Colpatria,negotiation,2021-05-19T14:54:46,6,17845868.0,76847


In [18]:
df_bajas = df_8658

In [19]:
df_bajas["inserted_at"] = pd.to_datetime(df_bajas["inserted_at"], format="mixed").dt.date

In [20]:
df_bajas

,id_berex,id,drop_date,financial_entity_name,status,inserted_at,active_debts,db,bank_reference
0,506867,1716699.0,2025-05-20,Banco de Bogota,new,2024-11-27,2,7933200.0,3024647576
1,506867,1716698.0,2025-05-20,Banco de Bogota,new,2024-11-27,2,2618000.0,3024647576
2,346434,1131486.0,2025-04-29,Davivienda,negotiation,2023-02-13,3,53603000.0,3102856558
3,346434,1131485.0,2025-04-29,Bancolombia,negotiation,2023-02-13,3,35426600.0,3102856558
4,346434,1131484.0,2025-04-29,Éxito,liquidation_structured_payment,2023-02-13,3,15042500.0,3102856558
...,...,...,...,...,...,...,...,...,...
313944,219613,721433.0,2021-10-31,Davivienda,negotiation,2021-05-19,6,891399.0,76847
313945,219613,721432.0,2021-10-31,Davivienda,negotiation,2021-05-19,6,483940.0,76847
313946,219613,721434.0,2021-10-31,Banco de Bogota,negotiation,2021-05-19,6,5447488.0,76847
313947,219613,721437.0,2021-10-31,Scotiabank Colpatria,negotiation,2021-05-19,6,17845868.0,76847


In [21]:
import os
import requests
import pandas as pd
from io import StringIO
from requests.exceptions import JSONDecodeError

BASE_URL = "https://metabase.resuelve.io"
CARD_ID = 7994  # resumen-clientes-deudas-clientes-baja


def running_in_colab():
    try:
        import google.colab  # noqa
        return True
    except ImportError:
        return False


def get_mb_credentials():
    user = None
    password = None

    if running_in_colab():
        from google.colab import userdata
        user = userdata.get("MB_USER")
        password = userdata.get("MB_PASS")

    if not user:
        user = os.environ.get("MB_USER")
    if not password:
        password = os.environ.get("MB_PASS")

    if not user or not password:
        raise ValueError("No encontré credenciales de Metabase.")

    return user, password


def create_metabase_session():
    user, password = get_mb_credentials()

    resp = requests.post(
        f"{BASE_URL}/api/session",
        json={"username": user, "password": password},
        timeout=60,
    )
    resp.raise_for_status()

    session_id = resp.json()["id"]
    headers = {"X-Metabase-Session": session_id}
    return headers


def get_card_full(card_id: int) -> pd.DataFrame:
    headers = create_metabase_session()

    try:
        resp = requests.post(
            f"{BASE_URL}/api/card/{card_id}/query/json",
            headers=headers,
            json={},
            timeout=600,
        )
        resp.raise_for_status()
        return pd.DataFrame(resp.json())

    except (JSONDecodeError, ValueError):
        resp = requests.post(
            f"{BASE_URL}/api/card/{card_id}/query/csv",
            headers=headers,
            json={},
            timeout=600,
        )
        resp.raise_for_status()
        return pd.read_csv(StringIO(resp.text))


df_7994 = get_card_full(CARD_ID)
print(df_7994.shape)
df_7994.head()

(75837, 14)


,id_berex,graduation_date,id,drop_date,financial_entity_name,fecha_liquidacion,status,inserted_at,liquidated_with_credit,mora_liq_o_hoy,db,paid_off_date,payment_to_bank,bank_reference
0,459671,None,1544479.0,None,Av Villas,2024-12-25,paid,2024-06-06T11:07:42,True,20.0,17908000.0,2024-12-26,6400000.0,3202894521
1,422700,None,1407358.0,None,Bancolombia,2024-09-18,paid,2024-01-09T11:57:10,True,10.0,29507500.0,2024-09-18,13250000.0,3004317677
2,417631,None,1388742.0,None,Banco de Bogota,None,negotiation,2023-12-11T18:20:12,False,25.0,976350.0,None,976350.0,3013428431
3,317356,None,1030574.0,None,Falabella,2023-06-26,paid,2022-09-28T17:42:02,True,9.0,35292667.0,2023-06-26,20500000.0,3143604542
4,436585,None,1458598.0,None,Éxito,2024-09-13,paid,2024-02-29T12:52:37,True,10.0,7576800.0,2024-09-13,2000000.0,3163607633


In [22]:
df_7994

,id_berex,graduation_date,id,drop_date,financial_entity_name,fecha_liquidacion,status,inserted_at,liquidated_with_credit,mora_liq_o_hoy,db,paid_off_date,payment_to_bank,bank_reference
0,459671,None,1544479.0,None,Av Villas,2024-12-25,paid,2024-06-06T11:07:42,True,20.0,17908000.0,2024-12-26,6400000.00,3202894521
1,422700,None,1407358.0,None,Bancolombia,2024-09-18,paid,2024-01-09T11:57:10,True,10.0,29507500.0,2024-09-18,13250000.00,3004317677
2,417631,None,1388742.0,None,Banco de Bogota,None,negotiation,2023-12-11T18:20:12,False,25.0,976350.0,None,976350.00,3013428431
3,317356,None,1030574.0,None,Falabella,2023-06-26,paid,2022-09-28T17:42:02,True,9.0,35292667.0,2023-06-26,20500000.00,3143604542
4,436585,None,1458598.0,None,Éxito,2024-09-13,paid,2024-02-29T12:52:37,True,10.0,7576800.0,2024-09-13,2000000.00,3163607633
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
75832,544535,None,1855316.0,None,Banco de Bogota,None,negotiation,2025-04-30T17:44:01,False,32.0,16667700.0,None,4426750.03,3194426919
75833,555860,None,1895916.0,None,Bancolombia,None,liquidation,2025-06-12T17:18:29,False,10.0,6642900.0,None,909000.00,3192699320
75834,498929,None,1687464.0,None,Banco de Bogota,None,negotiation,2024-10-24T18:28:51,False,15.0,24292800.0,None,9231706.19,3228157323
75835,532277,None,1809493.0,None,Alkosto,None,negotiation,2025-03-12T10:43:12,False,12.0,309350.0,None,257000.00,3102754205


In [23]:
df_activas = df_7994

#Base de mutatio

In [24]:
# ========= Imports =========
import io, os, requests, pandas as pd
from requests.adapters import HTTPAdapter
from urllib3.util.retry import Retry

# display solo si existe (Colab / notebook)
try:
    from IPython.display import display
except Exception:
    def display(x):
        print(x.head() if hasattr(x, "head") else x)

# ========= Secrets (Colab o ENV) =========
def _get_secret(name: str):
    try:
        from google.colab import userdata as _ud
        val = _ud.get(name)
        if val:
            return val
    except Exception:
        pass
    return os.environ.get(name)

USER_API   = _get_secret("USER_API")
SECRET_API = _get_secret("SECRET_API")
if not USER_API or not SECRET_API:
    raise ValueError("Falta USER_API o SECRET_API.")

# ========= Constantes =========
BASE_URL  = "https://mutatio-api.gobravo.dev"
RESOURCE  = "/accounting/repairs/download"

# ========= Sesión con reintentos =========
def make_session():
    s = requests.Session()
    retry = Retry(
        total=3, connect=2, read=2, backoff_factor=1.2,
        status_forcelist=[502,503,504], allowed_methods={"POST"},
        raise_on_status=False
    )
    s.mount("https://", HTTPAdapter(max_retries=retry))
    s.mount("http://",  HTTPAdapter(max_retries=retry))
    return s

session = make_session()

# ========= Auth =========
def get_token(user: str, secret: str) -> str:
    r = session.post(
        f"{BASE_URL}/auth/generate-token",
        json={"user": user, "secret": secret},
        headers={"Content-Type":"application/json","Accept":"application/json"},
        timeout=(10,45)
    )
    r.raise_for_status()
    tok = r.json().get("token")
    if not tok:
        raise RuntimeError("Auth OK pero no vino 'token'.")
    return tok

# ========= Util: leer CSV con ; o , =========
def _leer_csv_flexible(resp) -> pd.DataFrame:
    resp.raise_for_status()
    txt = resp.text or ""
    if not txt.strip():
        return pd.DataFrame()
    first = txt.splitlines()[0]
    sep = ';' if first.count(';') >= first.count(',') else ','
    return pd.read_csv(io.StringIO(txt), sep=sep, dtype=str)

# ========= Descarga =========
def descargar_pagina_repairs(user: str, secret: str, page: int, filtros=None) -> pd.DataFrame:
    if filtros is None:
        filtros = []
    url = f"{BASE_URL}{RESOURCE}"

    def _do(tok):
        return session.post(
            url,
            headers={
                "Authorization": f"Bearer {tok}",
                "Accept": "text/csv, text/plain",
                "Content-Type": "application/json",
            },
            params={"pageToDownload": str(page)},
            json=filtros,
            timeout=(12,90)
        )

    token = get_token(user, secret)
    r = _do(token)
    if r.status_code == 401:
        token = get_token(user, secret)
        r = _do(token)

    if r.status_code >= 400:
        raise RuntimeError(f"Error {r.status_code} en page={page}. {r.text[:300]}")

    return _leer_csv_flexible(r)

def descargar_todo_repairs(user: str, secret: str, max_pages: int = 1000, filtros=None) -> pd.DataFrame:
    frames = []
    for page in range(max_pages):
        df = descargar_pagina_repairs(user, secret, page=page, filtros=filtros)
        if df.empty:
            break
        frames.append(df)
    return pd.concat(frames, ignore_index=True) if frames else pd.DataFrame()

# ========= Ejecutar =========
reparadoras_df = descargar_todo_repairs(USER_API, SECRET_API, max_pages=1200)
print(f"Filas: {len(reparadoras_df)} | Columnas: {len(reparadoras_df.columns)}")
display(reparadoras_df.head())

Filas: 56139 | Columnas: 28


,Id,Referencia,Berex Id,Nombre Completo,Numero de Documento,Correo Electronico,Status,Fecha de Inicio,Tipo,Deuda Resuelve,...,Fecha de baja,Tipo de Pricing,Fecha de Reactivación,Porcentaje Liquidacion,Comision Inicial,Comision Mensual,Fecha de creacion,Fecha de actualizacion,Creado por,Actualizado por
0,231389,3187156050,597925,GERMAN ADOLFO CARDENAS LEON,79793894,gcard77@gmail.com,ACTIVO,11/12/2025,13,5.70253E7,...,NaN,Tradicional,NaN,0.15,1674022.0,295630.5,16/12/2025 13:50:36,16/12/2025 13:50:36,felipe.contreras,felipe.contreras
1,231380,3174057063,604143,ANDRES FELIPE SUAREZ CHISCO,1006862967,chiscoandres67@gmail.com,ACTIVO,9/12/2025,13,1.300156E7,...,NaN,Tradicional,NaN,0.15,643627.0,75511.8,16/12/2025 13:50:36,16/12/2025 13:50:36,felipe.contreras,felipe.contreras
2,231292,3003997262,604008,MARIA DEL ROSARIO BARRETO ESCOBAR,1115068936,mdrbarreto@hotmail.com,ACTIVO,8/12/2025,13,1.23255E7,...,NaN,Tradicional,NaN,0.15,805453.0,72131.5,15/12/2025 13:50:26,15/12/2025 13:50:26,felipe.contreras,felipe.contreras
3,231270,3126357252,603485,CARLOS DAVID AVILA DIAZ,1024473552,rocketcr7r10@hotmail.com,ACTIVO,9/12/2025,15,1.234552E8,...,NaN,Pricing,NaN,0.2,1474294.0,504324.8,14/12/2025 13:50:46,14/12/2025 13:50:46,felipe.contreras,felipe.contreras
4,231264,3222705973,603393,MARIA CAMILA PATIÑO MENDOZA,1110508761,lombi6757@gmail.com,ACTIVO,9/12/2025,13,1.20197E7,...,NaN,Tradicional,NaN,0.15,484828.0,70602.5,14/12/2025 13:50:46,14/12/2025 13:50:46,felipe.contreras,felipe.contreras


In [25]:
reparadora_inicial = reparadoras_df.copy()

#Construimos la base

###Reparadoras

In [26]:
columnas = ["Referencia", "Berex Id", "Status", "Fecha de Inicio", "Fecha de baja","Fecha de graduación", "Tipo de Pricing"]

reparadoras_df = reparadoras_df[columnas]

In [27]:
reparadoras_df

/usr/local/lib/python3.12/dist-packages/google/colab/_dataframe_summarizer.py:88: UserWarning: Parsing dates in %d/%m/%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  cast_date_col = pd.to_datetime(column, errors="coerce")
/usr/local/lib/python3.12/dist-packages/google/colab/_dataframe_summarizer.py:88: UserWarning: Parsing dates in %d/%m/%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  cast_date_col = pd.to_datetime(column, errors="coerce")


,Referencia,Berex Id,Status,Fecha de Inicio,Fecha de baja,Fecha de graduación,Tipo de Pricing
0,3187156050,597925,ACTIVO,11/12/2025,NaN,NaN,Tradicional
1,3174057063,604143,ACTIVO,9/12/2025,NaN,NaN,Tradicional
2,3003997262,604008,ACTIVO,8/12/2025,NaN,NaN,Tradicional
3,3126357252,603485,ACTIVO,9/12/2025,NaN,NaN,Pricing
4,3222705973,603393,ACTIVO,9/12/2025,NaN,NaN,Tradicional
...,...,...,...,...,...,...,...
56134,3167671311,6703,GRADUADO,25/1/2019,NaN,31/3/2024,Tradicional
56135,3023738871,6624,ESTRUCTURADO_INCUMPLIDO,23/1/2019,30/9/2023,NaN,Tradicional
56136,3133078002,6582,ESTRUCTURADO_INCUMPLIDO,22/1/2019,31/10/2023,NaN,Tradicional
56137,3046559962,6487,BAJA_2XC,18/1/2019,30/4/2025,NaN,Tradicional


In [28]:
referencia_buscar = 3193745298

resultado = reparadoras_df[reparadoras_df['Referencia'] == referencia_buscar]

resultado

,Referencia,Berex Id,Status,Fecha de Inicio,Fecha de baja,Fecha de graduación,Tipo de Pricing


In [29]:
def estandarizar_df(df):
    import pandas as pd
    import numpy as np

    df = df.copy()

    # ---------------------------------------------------------
    # 1. Convertir todas las columnas a MAYÚSCULA
    # ---------------------------------------------------------
    df.columns = df.columns.str.upper()

    # ---------------------------------------------------------
    # 2. Forzar REFERENCIA y BEREX ID a tipo object SOLO si existen
    # ---------------------------------------------------------
    for col in ["REFERENCIA", "BEREX ID"]:
        if col in df.columns:
            df[col] = df[col].astype(str)

    # ---------------------------------------------------------
    # 3. Detectar columnas de fecha por nombre o contenido
    # ---------------------------------------------------------
    def es_fecha(col):
        nombre = col.lower()
        palabras_fecha = ["fecha", "date", "inicio", "baja", "fin"]

        # Por nombre
        if any(p in nombre for p in palabras_fecha):
            return True

        # Por contenido
        try:
            pd.to_datetime(df[col], errors="raise")
            return True
        except:
            return False

    columnas_fecha = [col for col in df.columns if es_fecha(col)]

    # Convertirlas a datetime
    for col in columnas_fecha:
        df[col] = pd.to_datetime(df[col], errors="coerce")

    # ---------------------------------------------------------
    # 4. Detectar columnas numéricas y convertir a float
    # ---------------------------------------------------------
    for col in df.columns:
        if col not in columnas_fecha and col not in ["REFERENCIA", "BEREX ID"]:
            # Intentar convertir a número
            convertido = pd.to_numeric(df[col], errors="ignore")
            if pd.api.types.is_numeric_dtype(convertido):
                df[col] = convertido.astype(float)
            else:
                df[col] = df[col].astype(str)

    return df

In [30]:
reparadoras_df = estandarizar_df(reparadoras_df)

/tmp/ipython-input-1202366509.py:32: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  pd.to_datetime(df[col], errors="raise")
/tmp/ipython-input-1202366509.py:32: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  pd.to_datetime(df[col], errors="raise")
/tmp/ipython-input-1202366509.py:32: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  pd.to_datetime(df[col], errors="raise")
/tmp/ipython-input-1202366509.py:32: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify 

In [31]:
reparadoras_df

,REFERENCIA,BEREX ID,STATUS,FECHA DE INICIO,FECHA DE BAJA,FECHA DE GRADUACIÓN,TIPO DE PRICING
0,3187156050,597925,ACTIVO,2025-11-12,NaT,NaT,Tradicional
1,3174057063,604143,ACTIVO,2025-09-12,NaT,NaT,Tradicional
2,3003997262,604008,ACTIVO,2025-08-12,NaT,NaT,Tradicional
3,3126357252,603485,ACTIVO,2025-09-12,NaT,NaT,Pricing
4,3222705973,603393,ACTIVO,2025-09-12,NaT,NaT,Tradicional
...,...,...,...,...,...,...,...
56134,3167671311,6703,GRADUADO,NaT,NaT,2024-03-31,Tradicional
56135,3023738871,6624,ESTRUCTURADO_INCUMPLIDO,NaT,2023-09-30,NaT,Tradicional
56136,3133078002,6582,ESTRUCTURADO_INCUMPLIDO,NaT,2023-10-31,NaT,Tradicional
56137,3046559962,6487,BAJA_2XC,NaT,2025-04-30,NaT,Tradicional


In [32]:
reparadora_inicial.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 56139 entries, 0 to 56138
Data columns (total 28 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   Id                      56139 non-null  object
 1   Referencia              56139 non-null  object
 2   Berex Id                56139 non-null  object
 3   Nombre Completo         56139 non-null  object
 4   Numero de Documento     56139 non-null  object
 5   Correo Electronico      56139 non-null  object
 6   Status                  56139 non-null  object
 7   Fecha de Inicio         56139 non-null  object
 8   Tipo                    56139 non-null  object
 9   Deuda Resuelve          56139 non-null  object
 10  Movimientos Mensual     56139 non-null  object
 11  Numero de Deudas        56139 non-null  object
 12  Termino de programa     56139 non-null  object
 13  Vehiculo de ahorro      56139 non-null  object
 14  Empresa                 56139 non-null  object
 15  AR

# Agregamos liquidaciones desde 2020

In [33]:
df_liquidado_2025

,id_berex,graduation_date,debt_id,max,id,drop_date,financial_entity_name,inserted_at,liquidated_with_credit,db,paid_off_date,payment_to_bank,bank_reference
0,432204,None,1442113,2024-12-03,1442113,2025-03-25,Banco Popular,2024-02-14,False,42078.5,2024-12-03,12500.0000,3003925128
1,193957,None,645217,2022-01-07,645217,None,Scotiabank Colpatria,2021-01-12,True,16555.0,2021-12-24,8000.0000,46622
2,293068,2023-03-16,950756,2023-06-21,950756,None,Colsubsidio,2022-06-06,False,4983.0,2023-03-16,2420.0000,3014445964
3,395384,2024-07-22,1307194,2024-07-24,1307194,None,Rappipay,2023-09-08,True,28622.0,2024-07-22,19238.1300,1070623204
4,266503,2022-08-17,868155,2022-10-05,868155,None,Davivienda,2022-01-28,True,101346.0,2022-08-17,50000.0000,3193745298
...,...,...,...,...,...,...,...,...,...,...,...,...,...
69665,526578,2025-12-09,1788454,2025-12-10,1788454,None,BBVA,2025-02-18,True,150087.0,2025-12-09,53800.0000,3102895483
69666,581203,None,1989352,2025-12-10,1989352,None,Cobrando Falabella,2025-09-11,False,69883.0,2025-12-10,1318.0759,3227677307
69667,576219,None,1970886,2025-12-15,1970886,None,Sistecredito,2025-08-25,False,357.0,2025-12-15,374.5300,3106037019
69668,377169,None,1241912,2023-09-19,1241912,None,Sistecredito,2023-06-16,False,946.0,2023-09-11,1260.0600,3163298069


In [34]:
referencia_buscar = '3115518982'

resultado = df_liquidado_2025[df_liquidado_2025['bank_reference'] == referencia_buscar]

resultado

,id_berex,graduation_date,debt_id,max,id,drop_date,financial_entity_name,inserted_at,liquidated_with_credit,db,paid_off_date,payment_to_bank,bank_reference
22185,54328,2020-06-30,523854,2020-07-03,523854,None,Codensa,2020-07-03,True,34215.5,2020-07-03,75500.0,3115518982
30821,54328,2020-06-30,206385,2020-07-03,206385,None,Scotiabank Colpatria,2019-04-15,True,104870.0,2020-07-03,75500.0,3115518982


In [35]:
renombres = {
        "bank_reference": "REFERENCIA",
        "id_berex": "BEREX ID",
        "financial_entity_name": "BANCO",
        "liquidated_with_credit": "CREDITO",
        "db": "D_BRAVO",
        "paid_off_date": "FECHA DE PAGO",
        "payment_to_bank": "PAB"

    }

df_liquidado_2025 = df_liquidado_2025.rename(columns=renombres)


In [36]:
for col in ["REFERENCIA", "BEREX ID", "ID DEUDA"]:
    if col in df_liquidado_2025.columns:
         df_liquidado_2025[col] = df_liquidado_2025[col].astype(str)

In [37]:
import pandas as pd

# Asegurar que D_BRAVO es float
df_liquidado_2025["D_BRAVO"] = pd.to_numeric(df_liquidado_2025["D_BRAVO"], errors="coerce")

# Convertir FECHA DE PAGO a datetime
df_liquidado_2025["FECHA DE PAGO"] = pd.to_datetime(
    df_liquidado_2025["FECHA DE PAGO"],
    errors="coerce",       # Convierte fechas inválidas en NaT
    dayfirst=True          # Usa formato DD/MM/YYYY si viene así
)

df_liquidado_2025.dtypes

,0
BEREX ID,object
graduation_date,object
debt_id,int64
max,datetime64[ns]
id,int64
drop_date,object
BANCO,object
inserted_at,datetime64[ns]
CREDITO,bool
D_BRAVO,float64


In [38]:
df_liquidado_2025["D_BRAVO"] = df_liquidado_2025["D_BRAVO"] * 100
df_liquidado_2025["PAB"] = df_liquidado_2025["PAB"] * 100

In [39]:
df_liquidado_2025.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 69670 entries, 0 to 69669
Data columns (total 13 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   BEREX ID         69670 non-null  object        
 1   graduation_date  26916 non-null  object        
 2   debt_id          69670 non-null  int64         
 3   max              69670 non-null  datetime64[ns]
 4   id               69670 non-null  int64         
 5   drop_date        17414 non-null  object        
 6   BANCO            69570 non-null  object        
 7   inserted_at      69670 non-null  datetime64[ns]
 8   CREDITO          69670 non-null  bool          
 9   D_BRAVO          69670 non-null  float64       
 10  FECHA DE PAGO    69670 non-null  datetime64[ns]
 11  PAB              69670 non-null  float64       
 12  REFERENCIA       69670 non-null  object        
dtypes: bool(1), datetime64[ns](3), float64(2), int64(2), object(5)
memory usage: 6.4+ MB


In [40]:
df_liquidado_2025['max'] = pd.to_datetime(df_liquidado_2025['max'], errors='coerce')

df_liquidado_2025['max'] = df_liquidado_2025['max'].dt.date

df_liquidado_2025['FECHA DE PAGO'] = pd.to_datetime(df_liquidado_2025['FECHA DE PAGO'], errors='coerce')

df_liquidado_2025['FECHA DE PAGO'] = df_liquidado_2025['FECHA DE PAGO'].fillna(df_liquidado_2025['max'])


In [41]:
df_liquidado_2025

,BEREX ID,graduation_date,debt_id,max,id,drop_date,BANCO,inserted_at,CREDITO,D_BRAVO,FECHA DE PAGO,PAB,REFERENCIA
0,432204,None,1442113,2024-12-03,1442113,2025-03-25,Banco Popular,2024-02-14,False,4207850.0,2024-12-03,1250000.00,3003925128
1,193957,None,645217,2022-01-07,645217,None,Scotiabank Colpatria,2021-01-12,True,1655500.0,2021-12-24,800000.00,46622
2,293068,2023-03-16,950756,2023-06-21,950756,None,Colsubsidio,2022-06-06,False,498300.0,2023-03-16,242000.00,3014445964
3,395384,2024-07-22,1307194,2024-07-24,1307194,None,Rappipay,2023-09-08,True,2862200.0,2024-07-22,1923813.00,1070623204
4,266503,2022-08-17,868155,2022-10-05,868155,None,Davivienda,2022-01-28,True,10134600.0,2022-08-17,5000000.00,3193745298
...,...,...,...,...,...,...,...,...,...,...,...,...,...
69665,526578,2025-12-09,1788454,2025-12-10,1788454,None,BBVA,2025-02-18,True,15008700.0,2025-12-09,5380000.00,3102895483
69666,581203,None,1989352,2025-12-10,1989352,None,Cobrando Falabella,2025-09-11,False,6988300.0,2025-12-10,131807.59,3227677307
69667,576219,None,1970886,2025-12-15,1970886,None,Sistecredito,2025-08-25,False,35700.0,2025-12-15,37453.00,3106037019
69668,377169,None,1241912,2023-09-19,1241912,None,Sistecredito,2023-06-16,False,94600.0,2023-09-11,126006.00,3163298069


In [42]:
reparadoras_original = reparadoras_df.copy()

In [43]:
import pandas as pd
import numpy as np

# 1. Aseguramos tipos de fecha
df_liquidado_2025["max"] = pd.to_datetime(df_liquidado_2025["max"], errors="coerce")
df_liquidado_2025["FECHA DE PAGO"] = pd.to_datetime(df_liquidado_2025["FECHA DE PAGO"], errors="coerce")
reparadoras_df["FECHA DE INICIO"] = pd.to_datetime(reparadoras_df["FECHA DE INICIO"], errors="coerce")

# 2. Columnas que vamos a traer desde df_liquidado_2025
cols_liq = [
    "BEREX ID",
    "BANCO",
    "debt_id",
    "CREDITO",
    "D_BRAVO",
    "FECHA DE PAGO",
    "PAB",
    "max",
]

df_liq_subset = df_liquidado_2025[cols_liq].copy()

# 3. Primera FECHA DE INICIO por BEREX ID (para el caso max y FECHA DE PAGO nulas)
inicio_por_id = (
    reparadoras_df
    .groupby("BEREX ID", as_index=False)["FECHA DE INICIO"]
    .min()
    .rename(columns={"FECHA DE INICIO": "primera_fecha_inicio"})
)

# 4. Añadimos esa primera fecha a las deudas
df_liq_subset = df_liq_subset.merge(inicio_por_id, on="BEREX ID", how="left")

# 5. Construimos fecha_ref: max → FECHA DE PAGO → primera_fecha_inicio
df_liq_subset["fecha_ref"] = df_liq_subset["max"]

mask = df_liq_subset["fecha_ref"].isna()
df_liq_subset.loc[mask, "fecha_ref"] = df_liq_subset.loc[mask, "FECHA DE PAGO"]

mask = df_liq_subset["fecha_ref"].isna()
df_liq_subset.loc[mask, "fecha_ref"] = df_liq_subset.loc[mask, "primera_fecha_inicio"]

# --- AQUI CAMBIA LA LÓGICA: YA NO FILTRAMOS FILAS SIN fecha_ref ---
# Separamos las que tienen fecha_ref y las que no
df_liq_valid   = df_liq_subset[df_liq_subset["fecha_ref"].notna()].copy()
df_liq_missing = df_liq_subset[df_liq_subset["fecha_ref"].isna()].copy()

# 7. Preparamos reparadoras_df: ordenado por BEREX ID y FECHA DE INICIO y guardamos índice original
reparadoras_sorted = reparadoras_df.sort_values(
    ["BEREX ID", "FECHA DE INICIO"]
).reset_index()  # guarda índice original en columna "index"

reparadoras_sorted = reparadoras_sorted.rename(columns={"index": "orig_index"})

# 8. Creamos una columna para guardar el índice de la fila de reparadoras que le corresponde a cada deuda
df_liq_valid["match_index"] = pd.Series(index=df_liq_valid.index, dtype="float")
df_liq_missing["match_index"] = pd.Series(index=df_liq_missing.index, dtype="float")

# 9a. Para cada BEREX ID con fecha_ref, asignamos la fila correcta de reparadoras según la regla:
#     última FECHA DE INICIO <= fecha_ref (con clip para que siempre haya algo si existe FECHA DE INICIO)
for berex_id, grp_deuda in df_liq_valid.groupby("BEREX ID"):
    grp_rep = reparadoras_sorted[reparadoras_sorted["BEREX ID"] == berex_id]
    if grp_rep.empty:
        continue  # no hay info en reparadoras para este BEREX ID

    fechas_inicio = grp_rep["FECHA DE INICIO"].values
    fechas_ref = grp_deuda["fecha_ref"].values

    # posición donde se insertaría fecha_ref en fechas_inicio, por derecha
    idxs = np.searchsorted(fechas_inicio, fechas_ref, side="right") - 1

    # si queda -1 (fecha_ref antes de todas), la forzamos a 0 → toma la primera FECHA DE INICIO
    idxs = np.clip(idxs, 0, len(fechas_inicio) - 1)

    # mapeamos a los índices originales de reparadoras
    matched_orig_index = grp_rep["orig_index"].values[idxs]

    df_liq_valid.loc[grp_deuda.index, "match_index"] = matched_orig_index

# 9b. Para las filas SIN fecha_ref, aplicamos tu regla:
#     - si el BEREX ID tiene una sola FECHA DE INICIO distinta → usar esa
#     - si tiene 2+ fechas de inicio → usar la última FECHA DE INICIO
for berex_id, grp_deuda in df_liq_missing.groupby("BEREX ID"):
    grp_rep = reparadoras_sorted[reparadoras_sorted["BEREX ID"] == berex_id]
    if grp_rep.empty:
        continue  # no hay info en reparadoras para este BEREX ID

    # Fechas de inicio distintas, ignorando NaT
    fechas_unicas = grp_rep["FECHA DE INICIO"].dropna().unique()

    if len(fechas_unicas) == 0:
        # No hay ninguna fecha válida, no podemos asignar
        continue

    if len(fechas_unicas) == 1:
        # Solo una fecha de inicio → usamos esa
        fecha_target = fechas_unicas[0]
        # Por si hay varias filas con esa fecha, tomamos la última
        idx_rep = grp_rep[grp_rep["FECHA DE INICIO"] == fecha_target]["orig_index"].iloc[-1]
    else:
        # Varias fechas de inicio → usamos la última (la máxima)
        fecha_target = grp_rep["FECHA DE INICIO"].max()
        idx_rep = grp_rep.loc[grp_rep["FECHA DE INICIO"] == fecha_target, "orig_index"].iloc[-1]

    df_liq_missing.loc[grp_deuda.index, "match_index"] = idx_rep

# Unimos de nuevo valid + missing
df_liq_all = pd.concat([df_liq_valid, df_liq_missing], axis=0).sort_index()

# 10. Pasamos match_index a entero (puede tener NaN donde no se encontró nada)
df_liq_all["match_index"] = df_liq_all["match_index"].astype("Int64")

# 11. Preparamos reparadoras para el merge final por match_index
reparadoras_for_merge = reparadoras_df.copy()
reparadoras_for_merge["match_index"] = reparadoras_for_merge.index

# Evitamos duplicar columnas con el mismo nombre (preferimos las de df_liq_all para deuda)
overlap = set(df_liq_all.columns) & set(reparadoras_for_merge.columns)
overlap -= {"BEREX ID", "FECHA DE INICIO", "match_index"}  # estas sí las dejamos
if overlap:
    reparadoras_for_merge = reparadoras_for_merge.drop(columns=list(overlap))

# 12. Merge final: cada deuda con la fila de reparadoras que le corresponde
df_merged = df_liq_all.merge(
    reparadoras_for_merge,
    on="match_index",
    how="left"
)

# 13. Limpiamos columnas auxiliares
df_merged = df_merged.drop(columns=["primera_fecha_inicio", "match_index"], errors="ignore")

# 14. Este es tu reparadoras_df a nivel deuda, con la lógica de fechas aplicada
reparadoras_df = df_merged

In [44]:
reparadoras_df

,BEREX ID_x,BANCO,debt_id,CREDITO,D_BRAVO,FECHA DE PAGO,PAB,max,fecha_ref,REFERENCIA,BEREX ID_y,STATUS,FECHA DE INICIO,FECHA DE BAJA,FECHA DE GRADUACIÓN,TIPO DE PRICING
0,432204,Banco Popular,1442113,False,4207850.0,2024-12-03,1250000.00,2024-12-03,2024-12-03,3003925128,432204,BAJA_SOLICITADA,NaT,2025-03-31,NaT,Pricing
1,193957,Scotiabank Colpatria,645217,True,1655500.0,2021-12-24,800000.00,2022-01-07,2022-01-07,46622,193957,CREDITO_PARCIAL,NaT,2021-12-31,NaT,Tradicional
2,293068,Colsubsidio,950756,False,498300.0,2023-03-16,242000.00,2023-06-21,2023-06-21,NaN,NaN,NaN,NaT,NaT,NaT,NaN
3,395384,Rappipay,1307194,True,2862200.0,2024-07-22,1923813.00,2024-07-24,2024-07-24,1070623204,395384,GRADUADO_CREDITO,NaT,2024-04-30,2024-07-31,Tradicional
4,266503,Davivienda,868155,True,10134600.0,2022-08-17,5000000.00,2022-10-05,2022-10-05,NaN,NaN,NaN,NaT,NaT,NaT,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
69665,526578,BBVA,1788454,True,15008700.0,2025-12-09,5380000.00,2025-12-10,2025-12-10,3102895483,526578,CREDITO_PARCIAL,NaT,2025-08-31,NaT,Tradicional
69666,581203,Cobrando Falabella,1989352,False,6988300.0,2025-12-10,131807.59,2025-12-10,2025-12-10,3227677307,581203,ACTIVO,2025-10-09,NaT,NaT,Tradicional
69667,576219,Sistecredito,1970886,False,35700.0,2025-12-15,37453.00,2025-12-15,2025-12-15,3106037019,576219,ACTIVO,NaT,NaT,NaT,Tradicional
69668,377169,Sistecredito,1241912,False,94600.0,2023-09-11,126006.00,2023-09-19,2023-09-19,3163298069,377169,ACTIVO,NaT,NaT,NaT,Pricing Bravo


In [45]:
# En este punto:
# - reparadoras_df  == df_merged  (solo BEREX ID que vienen de df_liq_valid)
# - reparadoras_original tiene todas las filas originales

# 1. Aseguramos que ambas tengan una columna BEREX ID limpia
for df_name, df in [("reparadoras_df", reparadoras_df), ("reparadoras_original", reparadoras_original)]:
    cols = list(df.columns)
    if "BEREX ID" not in cols:
        if "BEREX ID_x" in cols and "BEREX ID_y" in cols:
            df["BEREX ID"] = df["BEREX ID_x"].fillna(df["BEREX ID_y"])
            df.drop(columns=["BEREX ID_x", "BEREX ID_y"], inplace=True, errors="ignore")
        elif "BEREX ID_x" in cols:
            df.rename(columns={"BEREX ID_x": "BEREX ID"}, inplace=True)
        elif "BEREX ID_y" in cols:
            df.rename(columns={"BEREX ID_y": "BEREX ID"}, inplace=True)

# 2. BEREX ID que ya están en el df de deudas (df_merged)
ids_con_deuda = reparadoras_df["BEREX ID"].dropna().unique()

# 3. Filas originales cuyo BEREX ID NO aparece en el df de deudas
faltantes = reparadoras_original[~reparadoras_original["BEREX ID"].isin(ids_con_deuda)].copy()

# 4. Aseguramos que faltantes tenga todas las columnas de reparadoras_df
for col in reparadoras_df.columns:
    if col not in faltantes.columns:
        faltantes[col] = np.nan

# Mismo orden de columnas
faltantes = faltantes[reparadoras_df.columns]

# 5. Concatenamos: deudas + filas originales sin deuda
reparadoras_df = pd.concat([reparadoras_df, faltantes], ignore_index=True)

/tmp/ipython-input-841683194.py:32: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  reparadoras_df = pd.concat([reparadoras_df, faltantes], ignore_index=True)


In [46]:
# 1. Renombrar 'debt_id' → 'ID DEUDA'
if "debt_id" in reparadoras_df.columns:
    reparadoras_df = reparadoras_df.rename(columns={"debt_id": "ID DEUDA"})

# 2. Definir el orden deseado
orden = [
    "BEREX ID",
    "REFERENCIA",
    "ID DEUDA",
    "BANCO",
    "FECHA DE INICIO",
    "FECHA DE PAGO",
    "FECHA DE BAJA",
    "FECHA DE GRADUACIÓN",
    "D_BRAVO",
    "PAB"
]

# 3. Asegurar que solo usamos columnas que existan en el df
orden_existente = [c for c in orden if c in reparadoras_df.columns]

# 4. Obtener el resto de columnas que van al final
resto = [c for c in reparadoras_df.columns if c not in orden_existente]

# 5. Reorganizar el DataFrame
reparadoras_df = reparadoras_df[orden_existente + resto]

In [47]:
for col in ["REFERENCIA", "BEREX ID", "ID DEUDA"]:
    if col in reparadoras_df.columns:
         reparadoras_df[col] = reparadoras_df[col].astype(str)

In [48]:
columnas_eliminar = ["max", "fecha_ref"]

reparadoras_df = reparadoras_df.drop(columns=columnas_eliminar)

In [49]:
reparadoras_df

,BEREX ID,REFERENCIA,ID DEUDA,BANCO,FECHA DE INICIO,FECHA DE PAGO,FECHA DE BAJA,FECHA DE GRADUACIÓN,D_BRAVO,PAB,CREDITO,STATUS,TIPO DE PRICING
0,432204,3003925128,1442113.0,Banco Popular,NaT,2024-12-03,2025-03-31,NaT,4207850.0,1250000.0,0.0,BAJA_SOLICITADA,Pricing
1,193957,46622,645217.0,Scotiabank Colpatria,NaT,2021-12-24,2021-12-31,NaT,1655500.0,800000.0,1.0,CREDITO_PARCIAL,Tradicional
2,293068,nan,950756.0,Colsubsidio,NaT,2023-03-16,NaT,NaT,498300.0,242000.0,0.0,NaN,NaN
3,395384,1070623204,1307194.0,Rappipay,NaT,2024-07-22,2024-04-30,2024-07-31,2862200.0,1923813.0,1.0,GRADUADO_CREDITO,Tradicional
4,266503,nan,868155.0,Davivienda,NaT,2022-08-17,NaT,NaT,10134600.0,5000000.0,1.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
106283,62319,3005290744,nan,NaN,NaT,NaT,2023-08-31,NaT,NaN,NaN,NaN,BAJA_2XC,Tradicional
106284,58880,3122962933,nan,NaN,NaT,NaT,NaT,NaT,NaN,NaN,NaN,ACTIVO,Tradicional
106285,55727,3113437460,nan,NaN,NaT,NaT,2023-07-31,NaT,NaN,NaN,NaN,BAJA_2XC,Tradicional
106286,53683,3108022384,nan,NaN,2019-11-04,NaT,2025-02-28,NaT,NaN,NaN,NaN,BAJA_2XC,Tradicional


#Bajas

In [50]:
renombres = {
        "bank_reference": "REFERENCIA",
        "id_berex": "BEREX ID",
        "financial_entity_name": "BANCO",
        "db": "D_BRAVO",
        "id": "ID DEUDA",

    }

df_bajas = df_bajas.rename(columns=renombres)

In [51]:
for col in ["REFERENCIA", "BEREX ID", "ID DEUDA"]:
    if col in df_bajas.columns:
         df_bajas[col] = df_bajas[col].astype(str)

In [ ]:
#df_bajas["D_BRAVO"] = df_bajas["D_BRAVO"] * 1000

In [52]:
df_bajas

,BEREX ID,ID DEUDA,drop_date,BANCO,status,inserted_at,active_debts,D_BRAVO,REFERENCIA
0,506867,1716699.0,2025-05-20,Banco de Bogota,new,2024-11-27,2,7933200.0,3024647576
1,506867,1716698.0,2025-05-20,Banco de Bogota,new,2024-11-27,2,2618000.0,3024647576
2,346434,1131486.0,2025-04-29,Davivienda,negotiation,2023-02-13,3,53603000.0,3102856558
3,346434,1131485.0,2025-04-29,Bancolombia,negotiation,2023-02-13,3,35426600.0,3102856558
4,346434,1131484.0,2025-04-29,Éxito,liquidation_structured_payment,2023-02-13,3,15042500.0,3102856558
...,...,...,...,...,...,...,...,...,...
313944,219613,721433.0,2021-10-31,Davivienda,negotiation,2021-05-19,6,891399.0,76847
313945,219613,721432.0,2021-10-31,Davivienda,negotiation,2021-05-19,6,483940.0,76847
313946,219613,721434.0,2021-10-31,Banco de Bogota,negotiation,2021-05-19,6,5447488.0,76847
313947,219613,721437.0,2021-10-31,Scotiabank Colpatria,negotiation,2021-05-19,6,17845868.0,76847


In [53]:
df_bajas.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 313949 entries, 0 to 313948
Data columns (total 9 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   BEREX ID      313949 non-null  object 
 1   ID DEUDA      313949 non-null  object 
 2   drop_date     313815 non-null  object 
 3   BANCO         313890 non-null  object 
 4   status        313948 non-null  object 
 5   inserted_at   313948 non-null  object 
 6   active_debts  313949 non-null  int64  
 7   D_BRAVO       313948 non-null  float64
 8   REFERENCIA    313949 non-null  object 
dtypes: float64(1), int64(1), object(7)
memory usage: 21.6+ MB


In [54]:
df_bajas.columns

Index(['BEREX ID', 'ID DEUDA', 'drop_date', 'BANCO', 'status', 'inserted_at',
       'active_debts', 'D_BRAVO', 'REFERENCIA'],
      dtype='object')

In [55]:
referencia_buscar = '383241'

resultado = df_bajas[df_bajas['BEREX ID'] == referencia_buscar]

resultado

,BEREX ID,ID DEUDA,drop_date,BANCO,status,inserted_at,active_debts,D_BRAVO,REFERENCIA
79173,383241,1263318.0,2023-07-31,Bancolombia,new,2023-07-15,1,17249400.0,3217466123


In [56]:
import numpy as np
import pandas as pd

# 1. IDs de deuda que ya existen en reparadoras_df
ids_existentes = set(
    reparadoras_df["ID DEUDA"].dropna().unique()
) if "ID DEUDA" in reparadoras_df.columns else set()

# 2. Filtrar df_bajas a las deudas nuevas (ID DEUDA que NO están en reparadoras_df)
df_bajas_nuevas = df_bajas[~df_bajas["ID DEUDA"].isin(ids_existentes)].copy()

# Si no hay nada nuevo, no hacemos nada
if not df_bajas_nuevas.empty:

    # 3. Construir plantilla por BEREX ID con la info de la referencia
    cols_plantilla = [
        "BEREX ID",
        "REFERENCIA",
        "FECHA DE INICIO",
        "FECHA DE BAJA",
        "STATUS",
        "TIPO DE PRICING"
    ]
    cols_plantilla_existentes = [c for c in cols_plantilla if c in reparadoras_df.columns]

    plantilla = (
        reparadoras_df
        .sort_values(["BEREX ID", "FECHA DE INICIO"])
        .groupby("BEREX ID", as_index=False)
        .tail(1)[cols_plantilla_existentes]
    )

    # 4. Armamos las filas nuevas a partir de df_bajas_nuevas
    #    Aquí INCLUIMOS REFERENCIA si viene en df_bajas
    cols_bajas = ["BEREX ID", "ID DEUDA", "D_BRAVO", "BANCO", "REFERENCIA"]
    cols_bajas_existentes = [c for c in cols_bajas if c in df_bajas_nuevas.columns]

    nuevas_filas = df_bajas_nuevas[cols_bajas_existentes].copy()

    # 5. Mezclamos con plantilla para copiar info de referencia
    nuevas_filas = nuevas_filas.merge(
        plantilla,
        on="BEREX ID",
        how="left",
        suffixes=("_baja", "_plantilla")
    )

    # 6. Resolver REFERENCIA: priorizar la de df_bajas, luego plantilla
    if "REFERENCIA_baja" in nuevas_filas.columns and "REFERENCIA_plantilla" in nuevas_filas.columns:
        nuevas_filas["REFERENCIA"] = nuevas_filas["REFERENCIA_baja"].fillna(
            nuevas_filas["REFERENCIA_plantilla"]
        )
        nuevas_filas = nuevas_filas.drop(columns=["REFERENCIA_baja", "REFERENCIA_plantilla"])
    elif "REFERENCIA_baja" in nuevas_filas.columns:
        nuevas_filas = nuevas_filas.rename(columns={"REFERENCIA_baja": "REFERENCIA"})
    elif "REFERENCIA_plantilla" in nuevas_filas.columns:
        nuevas_filas = nuevas_filas.rename(columns={"REFERENCIA_plantilla": "REFERENCIA"})
    # Si ninguna existe, se quedará como NaN y luego la creamos vacía más abajo.

    # 7. Aseguramos que nuevas_filas tenga TODAS las columnas de reparadoras_df.
    for col in reparadoras_df.columns:
        if col not in nuevas_filas.columns:
            nuevas_filas[col] = np.nan

    # Reordenamos columnas para que coincidan exactamente
    nuevas_filas = nuevas_filas[reparadoras_df.columns]

    # 8. Concatenamos al final de reparadoras_df
    reparadoras_df = pd.concat([reparadoras_df, nuevas_filas], ignore_index=True)

/tmp/ipython-input-385445283.py:69: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  reparadoras_df = pd.concat([reparadoras_df, nuevas_filas], ignore_index=True)
/tmp/ipython-input-385445283.py:69: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  reparadoras_df = pd.concat([reparadoras_df, nuevas_filas], ignore_index=True)


In [57]:
import pandas as pd

# 1) Asegurar tipos datetime
reparadoras_df["FECHA DE INICIO"] = pd.to_datetime(
    reparadoras_df["FECHA DE INICIO"],
    format="mixed",
    errors="coerce"
)

df_liquidado_2025["inserted_at"] = pd.to_datetime(
    df_liquidado_2025["inserted_at"],
    format="mixed",
    errors="coerce"
)

# (Opcional pero recomendado) Asegurar mismo tipo de BEREX ID en ambos DF
reparadoras_df["BEREX ID"] = reparadoras_df["BEREX ID"].astype(str)
df_liquidado_2025["BEREX ID"] = df_liquidado_2025["BEREX ID"].astype(str)

# 2) Crear un mapa BEREX ID -> inserted_at (una fila por BEREX ID)
#    Si hay varios registros por BEREX ID, se queda con el primero no nulo
map_inserted = (
    df_liquidado_2025
    .dropna(subset=["inserted_at"])
    .drop_duplicates(subset=["BEREX ID"])
    .set_index("BEREX ID")["inserted_at"]
)

# 3) Solo rellenar FECHA DE INICIO cuando es nula, usando inserted_at
mask_nula = reparadoras_df["FECHA DE INICIO"].isna()

reparadoras_df.loc[mask_nula, "FECHA DE INICIO"] = (
    reparadoras_df.loc[mask_nula, "BEREX ID"].map(map_inserted)
)

In [58]:
df_liquidado_2025

,BEREX ID,graduation_date,debt_id,max,id,drop_date,BANCO,inserted_at,CREDITO,D_BRAVO,FECHA DE PAGO,PAB,REFERENCIA
0,432204,None,1442113,2024-12-03,1442113,2025-03-25,Banco Popular,2024-02-14,False,4207850.0,2024-12-03,1250000.00,3003925128
1,193957,None,645217,2022-01-07,645217,None,Scotiabank Colpatria,2021-01-12,True,1655500.0,2021-12-24,800000.00,46622
2,293068,2023-03-16,950756,2023-06-21,950756,None,Colsubsidio,2022-06-06,False,498300.0,2023-03-16,242000.00,3014445964
3,395384,2024-07-22,1307194,2024-07-24,1307194,None,Rappipay,2023-09-08,True,2862200.0,2024-07-22,1923813.00,1070623204
4,266503,2022-08-17,868155,2022-10-05,868155,None,Davivienda,2022-01-28,True,10134600.0,2022-08-17,5000000.00,3193745298
...,...,...,...,...,...,...,...,...,...,...,...,...,...
69665,526578,2025-12-09,1788454,2025-12-10,1788454,None,BBVA,2025-02-18,True,15008700.0,2025-12-09,5380000.00,3102895483
69666,581203,None,1989352,2025-12-10,1989352,None,Cobrando Falabella,2025-09-11,False,6988300.0,2025-12-10,131807.59,3227677307
69667,576219,None,1970886,2025-12-15,1970886,None,Sistecredito,2025-08-25,False,35700.0,2025-12-15,37453.00,3106037019
69668,377169,None,1241912,2023-09-19,1241912,None,Sistecredito,2023-06-16,False,94600.0,2023-09-11,126006.00,3163298069


In [59]:
df_bajas

,BEREX ID,ID DEUDA,drop_date,BANCO,status,inserted_at,active_debts,D_BRAVO,REFERENCIA
0,506867,1716699.0,2025-05-20,Banco de Bogota,new,2024-11-27,2,7933200.0,3024647576
1,506867,1716698.0,2025-05-20,Banco de Bogota,new,2024-11-27,2,2618000.0,3024647576
2,346434,1131486.0,2025-04-29,Davivienda,negotiation,2023-02-13,3,53603000.0,3102856558
3,346434,1131485.0,2025-04-29,Bancolombia,negotiation,2023-02-13,3,35426600.0,3102856558
4,346434,1131484.0,2025-04-29,Éxito,liquidation_structured_payment,2023-02-13,3,15042500.0,3102856558
...,...,...,...,...,...,...,...,...,...
313944,219613,721433.0,2021-10-31,Davivienda,negotiation,2021-05-19,6,891399.0,76847
313945,219613,721432.0,2021-10-31,Davivienda,negotiation,2021-05-19,6,483940.0,76847
313946,219613,721434.0,2021-10-31,Banco de Bogota,negotiation,2021-05-19,6,5447488.0,76847
313947,219613,721437.0,2021-10-31,Scotiabank Colpatria,negotiation,2021-05-19,6,17845868.0,76847


In [60]:
import pandas as pd

# 1) Asegurar tipos datetime
reparadoras_df["FECHA DE INICIO"] = pd.to_datetime(
    reparadoras_df["FECHA DE INICIO"],
    format="mixed",
    errors="coerce"
)

df_bajas["inserted_at"] = pd.to_datetime(
    df_bajas["inserted_at"],
    format="mixed",
    errors="coerce"
)

# (Opcional pero recomendado) Asegurar mismo tipo de BEREX ID en ambos DF
reparadoras_df["BEREX ID"] = reparadoras_df["BEREX ID"].astype(str)
df_bajas["BEREX ID"] = df_bajas["BEREX ID"].astype(str)

# 2) Crear un mapa BEREX ID -> inserted_at (una fila por BEREX ID)
#    Si hay varios registros por BEREX ID, se queda con el primero no nulo
map_inserted = (
     df_bajas
    .dropna(subset=["inserted_at"])
    .drop_duplicates(subset=["BEREX ID"])
    .set_index("BEREX ID")["inserted_at"]
)

# 3) Solo rellenar FECHA DE INICIO cuando es nula, usando inserted_at
mask_nula = reparadoras_df["FECHA DE INICIO"].isna()

reparadoras_df.loc[mask_nula, "FECHA DE INICIO"] = (
    reparadoras_df.loc[mask_nula, "BEREX ID"].map(map_inserted)
)

In [61]:
import pandas as pd
import numpy as np

# Convertir a string para analizar el contenido
reparadoras_df["REFERENCIA"] = reparadoras_df["REFERENCIA"].astype(str)

# Crear máscara de valores válidos (solo dígitos puros)
mask_valida = reparadoras_df["REFERENCIA"].str.fullmatch(r"\d+")

# Lo que NO sean solo dígitos pasa a NaN
reparadoras_df.loc[~mask_valida, "REFERENCIA"] = np.nan

In [62]:
# Normalizar BEREX ID
for df in (reparadoras_df, df_bajas, df_liquidado_2025):
    df["BEREX ID"] = df["BEREX ID"].astype(str)

# Limpiar REFERENCIA en los DF de origen
for df in (df_bajas, df_liquidado_2025):
    df["REFERENCIA"] = df["REFERENCIA"].astype(str)
    mask_valida = df["REFERENCIA"].str.fullmatch(r"\d+")
    df.loc[~mask_valida, "REFERENCIA"] = np.nan

# Crear mapa con prioridad para df_liquidado_2025
map_referencia = (
    pd.concat([
        df_liquidado_2025[["BEREX ID", "REFERENCIA"]],
        df_bajas[["BEREX ID", "REFERENCIA"]],
    ], ignore_index=True)
    .dropna(subset=["REFERENCIA"])
    .drop_duplicates(subset=["BEREX ID"])
    .set_index("BEREX ID")["REFERENCIA"]
)


In [63]:
mask_ref_nula = reparadoras_df["REFERENCIA"].isna()

reparadoras_df.loc[mask_ref_nula, "REFERENCIA"] = (
    reparadoras_df.loc[mask_ref_nula, "BEREX ID"].map(map_referencia)
)

In [64]:
import numpy as np

# Aseguramos tipos string para hacer los joins/mapeos
for df in (reparadoras_df, df_bajas, df_liquidado_2025):
    df["BEREX ID"] = df["BEREX ID"].astype(str)
    df["REFERENCIA"] = df["REFERENCIA"].astype(str)

In [65]:
# Definimos qué BEREX ID consideramos inválidos
invalid_ids = {"0", "1", "10"}

# Construimos el mapa desde los dataframes de origen
ref_to_berex = (
    pd.concat([
        df_liquidado_2025[["REFERENCIA", "BEREX ID"]],
        df_bajas[["REFERENCIA", "BEREX ID"]],
    ], ignore_index=True)
    .assign(
        BEREX_ID_IS_INVALID=lambda d: d["BEREX ID"].isin(invalid_ids) | d["BEREX ID"].isna()
    )
    # Nos quedamos solo con BEREX ID válidos
    .loc[lambda d: ~d["BEREX_ID_IS_INVALID"]]
    # Si una REFERENCIA aparece varias veces, nos quedamos con la primera
    .drop_duplicates(subset=["REFERENCIA"])
    .set_index("REFERENCIA")["BEREX ID"]
)

In [66]:
# Máscara de BEREX ID inválido en reparadoras_df
mask_invalid_berex = (
    reparadoras_df["BEREX ID"].isin(invalid_ids) |
    reparadoras_df["BEREX ID"].isna()
)

# Actualizamos sólo esas filas, usando la REFERENCIA como clave
reparadoras_df.loc[mask_invalid_berex, "BEREX ID"] = (
    reparadoras_df.loc[mask_invalid_berex, "REFERENCIA"].map(ref_to_berex)
)

In [67]:
import pandas as pd

# 1) Asegurar tipos datetime en reparadoras_df
reparadoras_df["FECHA DE INICIO"] = pd.to_datetime(
    reparadoras_df["FECHA DE INICIO"],
    format="mixed",
    errors="coerce"
)

reparadoras_df["FECHA DE GRADUACIÓN"] = pd.to_datetime(
    reparadoras_df["FECHA DE GRADUACIÓN"],
    format="mixed",
    errors="coerce"
)

reparadoras_df["FECHA DE BAJA"] = pd.to_datetime(
    reparadoras_df["FECHA DE BAJA"],
    format="mixed",
    errors="coerce"
)

# 1b) Asegurar tipos datetime en df_liquidado_2025
df_liquidado_2025["inserted_at"] = pd.to_datetime(
    df_liquidado_2025["inserted_at"],
    format="mixed",
    errors="coerce"
)

df_liquidado_2025["graduation_date"] = pd.to_datetime(
    df_liquidado_2025["graduation_date"],
    format="mixed",
    errors="coerce"
)

df_liquidado_2025["drop_date"] = pd.to_datetime(
    df_liquidado_2025["drop_date"],
    format="mixed",
    errors="coerce"
)

# (Opcional pero recomendado) Asegurar mismo tipo de BEREX ID en ambos DF
reparadoras_df["BEREX ID"] = reparadoras_df["BEREX ID"].astype(str)
df_liquidado_2025["BEREX ID"] = df_liquidado_2025["BEREX ID"].astype(str)

# 2) Crear mapas BEREX ID -> fechas (una fila por BEREX ID)
#    inserted_at
map_inserted = (
    df_liquidado_2025
    .dropna(subset=["inserted_at"])
    .drop_duplicates(subset=["BEREX ID"])
    .set_index("BEREX ID")["inserted_at"]
)

#    graduation_date
map_graduation = (
    df_liquidado_2025
    .dropna(subset=["graduation_date"])
    .drop_duplicates(subset=["BEREX ID"])
    .set_index("BEREX ID")["graduation_date"]
)

#    drop_date
map_drop = (
    df_liquidado_2025
    .dropna(subset=["drop_date"])
    .drop_duplicates(subset=["BEREX ID"])
    .set_index("BEREX ID")["drop_date"]
)

# 3) Solo rellenar FECHA DE INICIO cuando es nula, usando inserted_at
mask_inicio_nula = reparadoras_df["FECHA DE INICIO"].isna()
reparadoras_df.loc[mask_inicio_nula, "FECHA DE INICIO"] = (
    reparadoras_df.loc[mask_inicio_nula, "BEREX ID"].map(map_inserted)
)

# 4) Solo rellenar FECHA DE GRADUACIÓN cuando es nula, usando graduation_date
mask_grad_nula = reparadoras_df["FECHA DE GRADUACIÓN"].isna()
reparadoras_df.loc[mask_grad_nula, "FECHA DE GRADUACIÓN"] = (
    reparadoras_df.loc[mask_grad_nula, "BEREX ID"].map(map_graduation)
)

# 5) Solo rellenar FECHA DE BAJA cuando es nula, usando drop_date
mask_baja_nula = reparadoras_df["FECHA DE BAJA"].isna()
reparadoras_df.loc[mask_baja_nula, "FECHA DE BAJA"] = (
    reparadoras_df.loc[mask_baja_nula, "BEREX ID"].map(map_drop)
)

In [68]:
import pandas as pd

# 1) Asegurar tipos datetime en reparadoras_df
reparadoras_df["FECHA DE INICIO"] = pd.to_datetime(
    reparadoras_df["FECHA DE INICIO"],
    format="mixed",
    errors="coerce"
)

reparadoras_df["FECHA DE BAJA"] = pd.to_datetime(
    reparadoras_df["FECHA DE BAJA"],
    format="mixed",
    errors="coerce"
)

# 1b) Asegurar tipos datetime en df_liquidado_2025
df_bajas["inserted_at"] = pd.to_datetime(
    df_bajas["inserted_at"],
    format="mixed",
    errors="coerce"
)


df_bajas["drop_date"] = pd.to_datetime(
    df_bajas["drop_date"],
    format="mixed",
    errors="coerce"
)

# (Opcional pero recomendado) Asegurar mismo tipo de BEREX ID en ambos DF
reparadoras_df["BEREX ID"] = reparadoras_df["BEREX ID"].astype(str)
df_bajas["BEREX ID"] = df_bajas["BEREX ID"].astype(str)

# 2) Crear mapas BEREX ID -> fechas (una fila por BEREX ID)
#    inserted_at
map_inserted = (
    df_bajas
    .dropna(subset=["inserted_at"])
    .drop_duplicates(subset=["BEREX ID"])
    .set_index("BEREX ID")["inserted_at"]
)


#    drop_date
map_drop = (
    df_bajas
    .dropna(subset=["drop_date"])
    .drop_duplicates(subset=["BEREX ID"])
    .set_index("BEREX ID")["drop_date"]
)

# 3) Solo rellenar FECHA DE INICIO cuando es nula, usando inserted_at
mask_inicio_nula = reparadoras_df["FECHA DE INICIO"].isna()
reparadoras_df.loc[mask_inicio_nula, "FECHA DE INICIO"] = (
    reparadoras_df.loc[mask_inicio_nula, "BEREX ID"].map(map_inserted)
)


# 5) Solo rellenar FECHA DE BAJA cuando es nula, usando drop_date
mask_baja_nula = reparadoras_df["FECHA DE BAJA"].isna()
reparadoras_df.loc[mask_baja_nula, "FECHA DE BAJA"] = (
    reparadoras_df.loc[mask_baja_nula, "BEREX ID"].map(map_drop)
)

In [69]:
df_bajas

,BEREX ID,ID DEUDA,drop_date,BANCO,status,inserted_at,active_debts,D_BRAVO,REFERENCIA
0,506867,1716699.0,2025-05-20,Banco de Bogota,new,2024-11-27,2,7933200.0,3024647576
1,506867,1716698.0,2025-05-20,Banco de Bogota,new,2024-11-27,2,2618000.0,3024647576
2,346434,1131486.0,2025-04-29,Davivienda,negotiation,2023-02-13,3,53603000.0,3102856558
3,346434,1131485.0,2025-04-29,Bancolombia,negotiation,2023-02-13,3,35426600.0,3102856558
4,346434,1131484.0,2025-04-29,Éxito,liquidation_structured_payment,2023-02-13,3,15042500.0,3102856558
...,...,...,...,...,...,...,...,...,...
313944,219613,721433.0,2021-10-31,Davivienda,negotiation,2021-05-19,6,891399.0,76847
313945,219613,721432.0,2021-10-31,Davivienda,negotiation,2021-05-19,6,483940.0,76847
313946,219613,721434.0,2021-10-31,Banco de Bogota,negotiation,2021-05-19,6,5447488.0,76847
313947,219613,721437.0,2021-10-31,Scotiabank Colpatria,negotiation,2021-05-19,6,17845868.0,76847


In [70]:
reparadoras_df

,BEREX ID,REFERENCIA,ID DEUDA,BANCO,FECHA DE INICIO,FECHA DE PAGO,FECHA DE BAJA,FECHA DE GRADUACIÓN,D_BRAVO,PAB,CREDITO,STATUS,TIPO DE PRICING
0,432204,3003925128,1442113.0,Banco Popular,2024-02-14,2024-12-03,2025-03-31,NaT,4207850.0,1250000.0,0.0,BAJA_SOLICITADA,Pricing
1,193957,46622,645217.0,Scotiabank Colpatria,2021-01-12,2021-12-24,2021-12-31,NaT,1655500.0,800000.0,1.0,CREDITO_PARCIAL,Tradicional
2,293068,3014445964,950756.0,Colsubsidio,2022-06-06,2023-03-16,NaT,2023-03-16,498300.0,242000.0,0.0,NaN,NaN
3,395384,1070623204,1307194.0,Rappipay,2023-09-08,2024-07-22,2024-04-30,2024-07-31,2862200.0,1923813.0,1.0,GRADUADO_CREDITO,Tradicional
4,266503,3193745298,868155.0,Davivienda,2022-01-28,2022-08-17,NaT,2022-08-17,10134600.0,5000000.0,1.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
404678,219613,76847,721433.0,Davivienda,2021-05-19,NaT,2021-10-31,NaT,891399.0,NaN,NaN,NaN,NaN
404679,219613,76847,721432.0,Davivienda,2021-05-19,NaT,2021-10-31,NaT,483940.0,NaN,NaN,NaN,NaN
404680,219613,76847,721434.0,Banco de Bogota,2021-05-19,NaT,2021-10-31,NaT,5447488.0,NaN,NaN,NaN,NaN
404681,219613,76847,721437.0,Scotiabank Colpatria,2021-05-19,NaT,2021-10-31,NaT,17845868.0,NaN,NaN,NaN,NaN


In [71]:
referencia_buscar = '3175157383'

resultado = reparadoras_df[reparadoras_df['REFERENCIA'] == referencia_buscar]

resultado

,BEREX ID,REFERENCIA,ID DEUDA,BANCO,FECHA DE INICIO,FECHA DE PAGO,FECHA DE BAJA,FECHA DE GRADUACIÓN,D_BRAVO,PAB,CREDITO,STATUS,TIPO DE PRICING
24934,431966,3175157383,1441186.0,Banco de Bogota,2024-02-14,2024-11-07,NaT,NaT,10091400.0,4479857.9,0.0,ACTIVO,Pricing
30338,431966,3175157383,1441183.0,Banco Finandina,2024-02-14,2025-05-29,NaT,NaT,5155700.0,3796000.0,0.0,ACTIVO,Pricing
32343,431966,3175157383,1441182.0,Éxito,2024-02-14,2024-08-29,NaT,NaT,1425600.0,746000.0,0.0,ACTIVO,Pricing
41187,431966,3175157383,1441184.0,Banco Finandina,2024-02-14,2025-05-29,NaT,NaT,7086200.0,5652000.0,0.0,ACTIVO,Pricing


In [72]:
reparadoras_df.replace(["nan", "NaN", "None", ""], pd.NA, inplace=True)

/tmp/ipython-input-2332946436.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  reparadoras_df.replace(["nan", "NaN", "None", ""], pd.NA, inplace=True)


In [73]:
import pandas as pd

# 1. Definimos las columnas que queremos completar desde df_bajas
cols_a_completar = ['ID DEUDA', 'BANCO', 'D_BRAVO']

# 2. Máscara: filas de reparadoras_df donde esas columnas son todas nulas
mask_nulos = reparadoras_df[cols_a_completar].isna().all(axis=1)

# 3. Separar las filas que NO necesitan cambio (las dejamos igual)
reparadoras_sin_cambios = reparadoras_df[~mask_nulos].copy()

# 4. Filas que SÍ necesitan buscar info en df_bajas
reparadoras_con_nulos = reparadoras_df[mask_nulos].copy()

# 5. Hacemos el merge SOLO con las filas con nulos
#    - Quitamos las columnas a completar para evitar conflictos
#    - Luego las traemos desde df_bajas
df_bajas_subset = df_bajas[['BEREX ID'] + cols_a_completar].copy()

reparadoras_completadas = (
    reparadoras_con_nulos
        .drop(columns=cols_a_completar)  # quitamos columnas vacías
        .merge(
            df_bajas_subset,
            on='BEREX ID',    # llave
            how='left'        # si no hay match, quedan NaN
        )
)

# 6. Unimos todo: las que no cambiaron + las nuevas filas completadas
reparadoras_df = pd.concat(
    [reparadoras_sin_cambios, reparadoras_completadas],
    ignore_index=True
)

In [74]:
# Trabajo sobre copia por seguridad (opcional)
reparadoras_df = reparadoras_df.copy()

# 1. Convertir a numérico lo que se pueda
id_numeric = pd.to_numeric(reparadoras_df['ID DEUDA'], errors='coerce')

# 2. Para los que NO son nulos: pasar a int y luego a string (sin .0)
id_str = id_numeric.dropna().astype(int).astype(str)

# 3. Asignar esos valores limpios de vuelta
reparadoras_df.loc[id_str.index, 'ID DEUDA'] = id_str

# 4. Aseguramos tipo object
reparadoras_df['ID DEUDA'] = reparadoras_df['ID DEUDA'].astype('object')

# 5. Alinear también el tipo de BEREX ID con df_activas (muy importante para el merge)
reparadoras_df['BEREX ID'] = reparadoras_df['BEREX ID'].astype(str)

In [75]:
reparadoras_df

,BEREX ID,REFERENCIA,ID DEUDA,BANCO,FECHA DE INICIO,FECHA DE PAGO,FECHA DE BAJA,FECHA DE GRADUACIÓN,D_BRAVO,PAB,CREDITO,STATUS,TIPO DE PRICING
0,432204,3003925128,1442113,Banco Popular,2024-02-14,2024-12-03,2025-03-31,NaT,4207850.00,1250000.0,0.0,BAJA_SOLICITADA,Pricing
1,193957,46622,645217,Scotiabank Colpatria,2021-01-12,2021-12-24,2021-12-31,NaT,1655500.00,800000.0,1.0,CREDITO_PARCIAL,Tradicional
2,293068,3014445964,950756,Colsubsidio,2022-06-06,2023-03-16,NaT,2023-03-16,498300.00,242000.0,0.0,NaN,NaN
3,395384,1070623204,1307194,Rappipay,2023-09-08,2024-07-22,2024-04-30,2024-07-31,2862200.00,1923813.0,1.0,GRADUADO_CREDITO,Tradicional
4,266503,3193745298,868155,Davivienda,2022-01-28,2022-08-17,NaT,2022-08-17,10134600.00,5000000.0,1.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
464472,55727,3113437460,210671,Scotiabank Colpatria,2019-04-27,NaT,2023-07-31,NaT,1654999.47,NaN,NaN,BAJA_2XC,Tradicional
464473,55727,3113437460,210673,Bancolombia,2019-04-27,NaT,2023-07-31,NaT,2341159.00,NaN,NaN,BAJA_2XC,Tradicional
464474,53683,3108022384,204324,Caja Social,2019-11-04,NaT,2025-02-28,NaT,9169800.00,NaN,NaN,BAJA_2XC,Tradicional
464475,37094,3029322,115231,Bancolombia,2019-02-27,NaT,2023-09-30,NaT,1245000.00,NaN,NaN,BAJA_2XC,Tradicional


#ACTIVAS

In [76]:
import pandas as pd

# Trabajo sobre copia por seguridad
df_activas = df_activas.copy()

# 1) Renombrar columnas
rename_map = {
    "id_berex": "BEREX ID",
    "graduation_date": "FECHA DE GRADUACIÓN",
    "id": "ID DEUDA",
    "drop_date": "FECHA DE BAJA",
    "financial_entity_name": "BANCO",
    "paid_off_date": "FECHA DE PAGO",
    "inserted_at": "FECHA DE INICIO",
    "liquidated_with_credit": "CREDITO",
    "db": "D_BRAVO",
    "payment_to_bank": "PAB",
    "bank_reference": "REFERENCIA",
}
df_activas = df_activas.rename(columns=rename_map)

# 2) BEREX ID → object (string)
if "BEREX ID" in df_activas.columns:
    df_activas["BEREX ID"] = df_activas["BEREX ID"].astype("string")
else:
    raise ValueError("No existe la columna 'BEREX ID' (venía de 'id_berex').")

# 3) ID DEUDA → object (string)
if "ID DEUDA" in df_activas.columns:
    df_activas["ID DEUDA"] = df_activas["ID DEUDA"].astype("string")
else:
    raise ValueError("No existe la columna 'ID DEUDA' (venía de 'id').")

# 4) Fechas a datetime (solo fecha, sin hora)
date_cols = ["FECHA DE GRADUACIÓN", "FECHA DE BAJA", "FECHA DE PAGO", "FECHA DE INICIO", "fecha_liquidacion"]
for col in date_cols:
    if col in df_activas.columns:
        df_activas[col] = pd.to_datetime(df_activas[col], errors="coerce").dt.normalize()

# 5) Si FECHA DE PAGO es nula y fecha_liquidacion no lo es, usar fecha_liquidacion
if "FECHA DE PAGO" in df_activas.columns and "fecha_liquidacion" in df_activas.columns:
    mask_pago_nulo = df_activas["FECHA DE PAGO"].isna() & df_activas["fecha_liquidacion"].notna()
    df_activas.loc[mask_pago_nulo, "FECHA DE PAGO"] = df_activas.loc[mask_pago_nulo, "fecha_liquidacion"]

# 6) Eliminar fecha_liquidacion si existe
if "fecha_liquidacion" in df_activas.columns:
    df_activas = df_activas.drop(columns=["fecha_liquidacion"])

# 7) CREDITO robusto: True/False/1/0/"true"/"false"/None -> 0/1
if "CREDITO" in df_activas.columns:
    s = df_activas["CREDITO"]

    # Normalizamos strings
    s = s.map(lambda x: x.strip().lower() if isinstance(x, str) else x)

    map_bool = {
        True: 1, False: 0,
        "true": 1, "false": 0,
        "1": 1, "0": 0,
        1: 1, 0: 0,
        "si": 1, "sí": 1, "no": 0,
        "yes": 1, "y": 1, "n": 0,
    }

    df_activas["CREDITO"] = (
        s.map(map_bool)      # convierte lo reconocido
         .fillna(0)          # lo desconocido/None/NaN -> 0 (conservador)
         .astype("int64")    # seguro
    )
else:
    raise ValueError("No existe la columna 'CREDITO' (venía de 'liquidated_with_credit').")

# (Opcional) asegurar numéricos si quieres:
# for c in ["D_BRAVO", "PAB"]:
#     if c in df_activas.columns:
#         df_activas[c] = pd.to_numeric(df_activas[c], errors="coerce")

# Listo: df_activas queda estandarizado

In [77]:
df_activas

,BEREX ID,FECHA DE GRADUACIÓN,ID DEUDA,FECHA DE BAJA,BANCO,status,FECHA DE INICIO,CREDITO,mora_liq_o_hoy,D_BRAVO,FECHA DE PAGO,PAB,REFERENCIA
0,459671,NaT,1544479.0,NaT,Av Villas,paid,2024-06-06,1,20.0,17908000.0,2024-12-26,6400000.00,3202894521
1,422700,NaT,1407358.0,NaT,Bancolombia,paid,2024-01-09,1,10.0,29507500.0,2024-09-18,13250000.00,3004317677
2,417631,NaT,1388742.0,NaT,Banco de Bogota,negotiation,2023-12-11,0,25.0,976350.0,NaT,976350.00,3013428431
3,317356,NaT,1030574.0,NaT,Falabella,paid,2022-09-28,1,9.0,35292667.0,2023-06-26,20500000.00,3143604542
4,436585,NaT,1458598.0,NaT,Éxito,paid,2024-02-29,1,10.0,7576800.0,2024-09-13,2000000.00,3163607633
...,...,...,...,...,...,...,...,...,...,...,...,...,...
75832,544535,NaT,1855316.0,NaT,Banco de Bogota,negotiation,2025-04-30,0,32.0,16667700.0,NaT,4426750.03,3194426919
75833,555860,NaT,1895916.0,NaT,Bancolombia,liquidation,2025-06-12,0,10.0,6642900.0,NaT,909000.00,3192699320
75834,498929,NaT,1687464.0,NaT,Banco de Bogota,negotiation,2024-10-24,0,15.0,24292800.0,NaT,9231706.19,3228157323
75835,532277,NaT,1809493.0,NaT,Alkosto,negotiation,2025-03-12,0,12.0,309350.0,NaT,257000.00,3102754205


In [78]:
reparadoras_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 464477 entries, 0 to 464476
Data columns (total 13 columns):
 #   Column               Non-Null Count   Dtype         
---  ------               --------------   -----         
 0   BEREX ID             464477 non-null  object        
 1   REFERENCIA           464454 non-null  object        
 2   ID DEUDA             454817 non-null  object        
 3   BANCO                454672 non-null  object        
 4   FECHA DE INICIO      455909 non-null  datetime64[ns]
 5   FECHA DE PAGO        69670 non-null   datetime64[ns]
 6   FECHA DE BAJA        422357 non-null  datetime64[ns]
 7   FECHA DE GRADUACIÓN  31478 non-null   datetime64[ns]
 8   D_BRAVO              454816 non-null  float64       
 9   PAB                  69670 non-null   float64       
 10  CREDITO              69670 non-null   float64       
 11  STATUS               249653 non-null  object        
 12  TIPO DE PRICING      249653 non-null  object        
dtypes: datetime64[

In [ ]:
df_activas.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 76187 entries, 0 to 76186
Data columns (total 13 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   BEREX ID             76187 non-null  object        
 1   FECHA DE GRADUACIÓN  243 non-null    datetime64[ns]
 2   ID DEUDA             76187 non-null  object        
 3   FECHA DE BAJA        1051 non-null   datetime64[ns]
 4   BANCO                76187 non-null  object        
 5   status               76187 non-null  object        
 6   FECHA DE INICIO      76187 non-null  datetime64[ns]
 7   CREDITO              76187 non-null  int64         
 8   mora_liq_o_hoy       76187 non-null  int64         
 9   D_BRAVO              76187 non-null  float64       
 10  FECHA DE PAGO        22692 non-null  datetime64[ns]
 11  PAB                  75462 non-null  float64       
 12  REFERENCIA           76187 non-null  object        
dtypes: datetime64[ns](4), float64(2

In [ ]:
import pandas as pd

# ======================================================
# 1. LIMPIEZA Y TRANSFORMACIÓN DE df_activas
# ======================================================

df_activas = df_activas.copy()

# Renombrar columnas
df_activas = df_activas.rename(columns={
    'id_berex': 'BEREX ID',
    'graduation_date': 'FECHA DE GRADUACIÓN',
    'id': 'ID DEUDA',
    'drop_date': 'FECHA DE BAJA',
    'financial_entity_name': 'BANCO',
    'paid_off_date': 'FECHA DE PAGO',
    'inserted_at': 'FECHA DE INICIO',
    'liquidated_with_credit': 'CREDITO',
    'db': 'D_BRAVO',
    'payment_to_bank': 'PAB',
    'bank_reference': 'REFERENCIA'
})

# BEREX ID e ID DEUDA como texto
df_activas['BEREX ID'] = df_activas['BEREX ID'].astype(str)
df_activas['ID DEUDA'] = df_activas['ID DEUDA'].astype(str)

# Convertir fechas a datetime y dejar solo fecha (sin hora)
for col in ['FECHA DE GRADUACIÓN', 'FECHA DE BAJA', 'FECHA DE PAGO', 'FECHA DE INICIO']:
    df_activas[col] = pd.to_datetime(df_activas[col], errors='coerce')
    df_activas[col] = df_activas[col].dt.normalize()

# CREDITO como entero (0/1)
df_activas['CREDITO'] = df_activas['CREDITO'].astype(int)

# ======================================================
# 2. LIMPIEZA DE reparadoras_df (ID DEUDA, BEREX ID, CREDITO)
# ======================================================

reparadoras_df = reparadoras_df.copy()

# Limpiar ID DEUDA en reparadoras_df (quitar .0)
id_numeric = pd.to_numeric(reparadoras_df['ID DEUDA'], errors='coerce')
id_str = id_numeric.dropna().astype('Int64').astype(str)
reparadoras_df.loc[id_str.index, 'ID DEUDA'] = id_str
reparadoras_df['ID DEUDA'] = reparadoras_df['ID DEUDA'].astype('object')

# Asegurar que BEREX ID sea texto
reparadoras_df['BEREX ID'] = reparadoras_df['BEREX ID'].astype(str)

# CREDITO como entero con soporte para NaN
if 'CREDITO' in reparadoras_df.columns:
    reparadoras_df['CREDITO'] = reparadoras_df['CREDITO'].astype('Int64')

# ======================================================
# 3. COMPLETAR reparadoras_df DESDE df_bajas
#    (ID DEUDA, BANCO, D_BRAVO)
# ======================================================

cols_bajas = ['ID DEUDA', 'BANCO', 'D_BRAVO']

# Filas donde esas columnas están TODAS nulas
mask_nulos_bajas = reparadoras_df[cols_bajas].isna().all(axis=1)

reparadoras_sin_cambios_bajas = reparadoras_df[~mask_nulos_bajas].copy()
reparadoras_con_nulos_bajas = reparadoras_df[mask_nulos_bajas].copy()

# Subset de df_bajas
df_bajas_subset = df_bajas[['BEREX ID'] + cols_bajas].copy()
df_bajas_subset['BEREX ID'] = df_bajas_subset['BEREX ID'].astype(str)

# Merge: si BEREX ID se repite en df_bajas, se crean varias filas
reparadoras_desde_bajas = (
    reparadoras_con_nulos_bajas
        .drop(columns=cols_bajas)
        .merge(df_bajas_subset, on='BEREX ID', how='left')
)

# Resultado intermedio después de usar df_bajas
reparadoras_df_bajas = pd.concat(
    [reparadoras_sin_cambios_bajas, reparadoras_desde_bajas],
    ignore_index=True
)

# ======================================================
# 4. COMPLETAR reparadoras_df_bajas DESDE df_activas
#    (FECHA DE GRADUACIÓN, ID DEUDA, FECHA DE BAJA, BANCO,
#     FECHA DE INICIO, CREDITO, D_BRAVO, FECHA DE PAGO, PAB, REFERENCIA)
# ======================================================

cols_activas = [
    'FECHA DE GRADUACIÓN',
    'ID DEUDA',
    'FECHA DE BAJA',
    'BANCO',
    'FECHA DE INICIO',
    'CREDITO',
    'D_BRAVO',
    'FECHA DE PAGO',
    'PAB',
    'REFERENCIA'
]

# Filas donde TODAS esas columnas están nulas
mask_nulos_activas = reparadoras_df_bajas[cols_activas].isna().all(axis=1)

rep_sin_cambios_act = reparadoras_df_bajas[~mask_nulos_activas].copy()
rep_con_nulos_act = reparadoras_df_bajas[mask_nulos_activas].copy()

# Subset de df_activas
df_activas_subset = df_activas[['BEREX ID'] + cols_activas].copy()

# Merge: si BEREX ID se repite en df_activas, se crean varias filas
rep_desde_activas = (
    rep_con_nulos_act
        .drop(columns=cols_activas)
        .merge(df_activas_subset, on='BEREX ID', how='left')
)

# DataFrame final
reparadoras_df_final = pd.concat(
    [rep_sin_cambios_act, rep_desde_activas],
    ignore_index=True
)

# Aseguramos CREDITO como entero al final
reparadoras_df_final['CREDITO'] = reparadoras_df_final['CREDITO'].astype('Int64')

In [ ]:
reparadoras_df = reparadoras_df_final.copy()

In [ ]:
import pandas as pd
import numpy as np

df = reparadoras_df.copy()

# 1. Identificar para cada REFERENCIA el BEREX ID mayoritario (el que más aparece)
berex_mayoritario = (
    df.groupby("REFERENCIA")["BEREX ID"]
      .agg(lambda x: x.mode().iloc[0] if not x.mode().empty else np.nan)
      .rename("BEREX_ID_MAYORITARIO")
)

# 2. Unir esta info al dataframe
df = df.merge(berex_mayoritario, on="REFERENCIA", how="left")

# 3. Crear condición para detectar BEREX ID distinto
df["BEREX_DISTINTO"] = df["BEREX ID"] != df["BEREX_ID_MAYORITARIO"]

# 4. Condición para eliminar:
#    - BEREX ID distinto
#    - AND ID DEUDA es NaN
mask_eliminar = (df["BEREX_DISTINTO"]) & (df["ID DEUDA"].isna())

# 5. Filtrar
df_filtrado = df[~mask_eliminar].copy()

print("Filas originales:", len(df))
print("Filas eliminadas:", mask_eliminar.sum())
print("Filas finales:", len(df_filtrado))

# Opcional: eliminar columnas auxiliares
df_filtrado = df_filtrado.drop(columns=["BEREX_ID_MAYORITARIO", "BEREX_DISTINTO"])

df_filtrado.head()

Filas originales: 463473
Filas eliminadas: 365
Filas finales: 463108


,BEREX ID,REFERENCIA,ID DEUDA,BANCO,FECHA DE INICIO,FECHA DE PAGO,FECHA DE BAJA,FECHA DE GRADUACIÓN,D_BRAVO,PAB,CREDITO,STATUS,TIPO DE PRICING
0,432204,3003925128,1442113,Banco Popular,2024-02-14,2024-12-03,2025-03-31,NaT,4207850.0,1250000.0,0,BAJA_SOLICITADA,Pricing
1,193957,46622,645217,Scotiabank Colpatria,2021-01-12,2021-12-24,2021-12-31,NaT,1655500.0,800000.0,1,CREDITO_PARCIAL,Tradicional
2,293068,3014445964,950756,Colsubsidio,2022-06-06,2023-03-16,NaT,2023-03-16,498300.0,242000.0,0,NaN,NaN
3,395384,1070623204,1307194,Rappipay,2023-09-08,2024-07-22,2024-04-30,2024-07-31,2862200.0,1923813.0,1,GRADUADO_CREDITO,Tradicional
4,266503,3193745298,868155,Davivienda,2022-01-28,2022-08-17,NaT,2022-08-17,10134600.0,5000000.0,1,NaN,NaN


In [ ]:
reparadoras_df = df_filtrado.copy()

In [ ]:
reparadoras_df

,BEREX ID,REFERENCIA,ID DEUDA,BANCO,FECHA DE INICIO,FECHA DE PAGO,FECHA DE BAJA,FECHA DE GRADUACIÓN,D_BRAVO,PAB,CREDITO,STATUS,TIPO DE PRICING
0,432204,3003925128,1442113,Banco Popular,2024-02-14,2024-12-03,2025-03-31,NaT,4207850.0,1250000.0,0,BAJA_SOLICITADA,Pricing
1,193957,46622,645217,Scotiabank Colpatria,2021-01-12,2021-12-24,2021-12-31,NaT,1655500.0,800000.0,1,CREDITO_PARCIAL,Tradicional
2,293068,3014445964,950756,Colsubsidio,2022-06-06,2023-03-16,NaT,2023-03-16,498300.0,242000.0,0,NaN,NaN
3,395384,1070623204,1307194,Rappipay,2023-09-08,2024-07-22,2024-04-30,2024-07-31,2862200.0,1923813.0,1,GRADUADO_CREDITO,Tradicional
4,266503,3193745298,868155,Davivienda,2022-01-28,2022-08-17,NaT,2022-08-17,10134600.0,5000000.0,1,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
463468,177016,3142625797,NaN,NaN,NaT,NaT,NaT,NaT,NaN,NaN,<NA>,ACTIVO,Tradicional
463469,173778,3213436450,NaN,NaN,NaT,NaT,NaT,NaT,NaN,NaN,<NA>,ACTIVO,Tradicional
463470,172169,3058143783,NaN,NaN,NaT,NaT,NaT,NaT,NaN,NaN,<NA>,ACTIVO,Tradicional
463471,123911,3183092531,NaN,NaN,NaT,NaT,NaT,NaT,NaN,NaN,<NA>,ACTIVO,Tradicional


In [ ]:
df_activas.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 76187 entries, 0 to 76186
Data columns (total 13 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   BEREX ID             76187 non-null  object        
 1   FECHA DE GRADUACIÓN  243 non-null    datetime64[ns]
 2   ID DEUDA             76187 non-null  object        
 3   FECHA DE BAJA        1051 non-null   datetime64[ns]
 4   BANCO                76187 non-null  object        
 5   status               76187 non-null  object        
 6   FECHA DE INICIO      76187 non-null  datetime64[ns]
 7   CREDITO              76187 non-null  int64         
 8   mora_liq_o_hoy       76187 non-null  int64         
 9   D_BRAVO              76187 non-null  float64       
 10  FECHA DE PAGO        22692 non-null  datetime64[ns]
 11  PAB                  75462 non-null  float64       
 12  REFERENCIA           76187 non-null  object        
dtypes: datetime64[ns](4), float64(2

In [ ]:
reparadoras_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 463108 entries, 0 to 463472
Data columns (total 13 columns):
 #   Column               Non-Null Count   Dtype         
---  ------               --------------   -----         
 0   BEREX ID             463108 non-null  object        
 1   REFERENCIA           463086 non-null  object        
 2   ID DEUDA             453598 non-null  object        
 3   BANCO                453453 non-null  object        
 4   FECHA DE INICIO      454658 non-null  datetime64[ns]
 5   FECHA DE PAGO        69508 non-null   datetime64[ns]
 6   FECHA DE BAJA        421099 non-null  datetime64[ns]
 7   FECHA DE GRADUACIÓN  31431 non-null   datetime64[ns]
 8   D_BRAVO              453597 non-null  float64       
 9   PAB                  69508 non-null   float64       
 10  CREDITO              69508 non-null   Int64         
 11  STATUS               248288 non-null  object        
 12  TIPO DE PRICING      248288 non-null  object        
dtypes: Int64(1), dateti

### agregar de activas lo que quedo faltando

In [ ]:
import numpy as np
import pandas as pd

# --- 1. Normalizamos nulos en ID DEUDA (por si hay strings 'nan') ---
reparadoras_df['ID DEUDA'] = reparadoras_df['ID DEUDA'].replace(
    ['nan', 'NaN', 'None', ''], np.nan
)

# --- 2. Definimos columnas y nombres clave ---
cols_actualizar_todas = [
    'BANCO',
    'D_BRAVO',
    'PAB',
    'CREDITO',
    'FECHA DE INICIO',
    'FECHA DE PAGO',
    'FECHA DE BAJA',
    'FECHA DE GRADUACIÓN'
]

col_id_deuda = 'ID DEUDA'
col_berex = 'BEREX ID'
col_ref = 'REFERENCIA'

# --- 3. Primero: actualizar y duplicar filas cuando ID DEUDA es nulo en reparadoras_df ---

mask_id_nulo = reparadoras_df[col_id_deuda].isna()
idxs_a_buscar = reparadoras_df.index[mask_id_nulo]

nuevas_filas = []  # filas nuevas que se irán agregando

for idx in idxs_a_buscar:
    fila_rep = reparadoras_df.loc[idx]
    berex_val = fila_rep[col_berex]
    ref_val = fila_rep[col_ref]

    # 3.1. Buscar primero por BEREX ID
    matches = df_activas[df_activas[col_berex] == berex_val]

    # 3.2. Si no encontró por BEREX ID, buscar por REFERENCIA
    if matches.empty:
        matches = df_activas[df_activas[col_ref] == ref_val]

    # 3.3. Si tampoco encontró nada, pasar a la siguiente fila
    if matches.empty:
        continue

    # --- 4. Usar el primer match para actualizar la fila original ---
    match_principal = matches.iloc[0]

    # 4.1. Rellenar ID DEUDA si está nulo y en df_activas sí tiene valor
    if pd.isna(fila_rep[col_id_deuda]) and not pd.isna(match_principal[col_id_deuda]):
        reparadoras_df.at[idx, col_id_deuda] = match_principal[col_id_deuda]

    # 4.2. Actualizar el resto de columnas si son diferentes o nulas en reparadoras_df
    for col in cols_actualizar_todas:
        if col in df_activas.columns:
            val_activa = match_principal[col]
            if not pd.isna(val_activa):  # solo si df_activas tiene dato
                val_rep = fila_rep[col] if col in reparadoras_df.columns else np.nan
                # si está nulo o es diferente, lo reemplazamos
                if (pd.isna(val_rep)) or (val_rep != val_activa):
                    reparadoras_df.at[idx, col] = val_activa

    # --- 5. Si hay más de un match, crear filas nuevas en reparadoras_df ---
    if len(matches) > 1:
        base_row = reparadoras_df.loc[idx].copy()  # ya actualizada

        for _, match_extra in matches.iloc[1:].iterrows():
            nueva_fila = base_row.copy()

            # ID DEUDA
            if not pd.isna(match_extra[col_id_deuda]):
                nueva_fila[col_id_deuda] = match_extra[col_id_deuda]

            # Otras columnas
            for col in cols_actualizar_todas:
                if col in df_activas.columns:
                    val_extra = match_extra[col]
                    if not pd.isna(val_extra):
                        nueva_fila[col] = val_extra

            nuevas_filas.append(nueva_fila)

# --- 6. Concatenar las nuevas filas de múltiples matches (si hay) ---
if nuevas_filas:
    nuevas_filas_df = pd.DataFrame(nuevas_filas)
    reparadoras_df = pd.concat([reparadoras_df, nuevas_filas_df], ignore_index=True)


# --- 7. Segundo: agregar filas que existen en df_activas pero NO en reparadoras_df ---

# Conjuntos de BEREX ID y REFERENCIA ya presentes en reparadoras_df
set_berex_rep = set(reparadoras_df[col_berex].dropna().unique())
set_ref_rep = set(reparadoras_df[col_ref].dropna().unique())

nuevas_filas_desde_activas = []

for _, fila_act in df_activas.iterrows():
    berex_val = fila_act[col_berex]
    ref_val = fila_act[col_ref]

    # ¿Ya existe en reparadoras_df por BEREX ID o por REFERENCIA?
    ya_existe = False
    if not pd.isna(berex_val) and berex_val in set_berex_rep:
        ya_existe = True
    if not pd.isna(ref_val) and ref_val in set_ref_rep:
        ya_existe = True

    if ya_existe:
        continue  # no creamos fila nueva; ya está representado

    # Si NO existe en reparadoras_df, creamos una nueva fila
    nueva_fila = {}

    for col in reparadoras_df.columns:
        if col in df_activas.columns:
            nueva_fila[col] = fila_act[col]  # usamos lo que tenga df_activas
        else:
            nueva_fila[col] = np.nan         # columnas que solo existen en reparadoras_df quedan nulas

    nuevas_filas_desde_activas.append(nueva_fila)

# --- 8. Concatenar estas filas nuevas (si hay) ---
if nuevas_filas_desde_activas:
    nuevas_desde_activas_df = pd.DataFrame(nuevas_filas_desde_activas)
    reparadoras_df = pd.concat([reparadoras_df, nuevas_desde_activas_df], ignore_index=True)

In [ ]:
reparadoras_df

,BEREX ID,REFERENCIA,ID DEUDA,BANCO,FECHA DE INICIO,FECHA DE PAGO,FECHA DE BAJA,FECHA DE GRADUACIÓN,D_BRAVO,PAB,CREDITO,STATUS,TIPO DE PRICING
0,432204,3003925128,1442113,Banco Popular,2024-02-14,2024-12-03,2025-03-31,NaT,4207850.0,1250000.0,0,BAJA_SOLICITADA,Pricing
1,193957,46622,645217,Scotiabank Colpatria,2021-01-12,2021-12-24,2021-12-31,NaT,1655500.0,800000.0,1,CREDITO_PARCIAL,Tradicional
2,293068,3014445964,950756,Colsubsidio,2022-06-06,2023-03-16,NaT,2023-03-16,498300.0,242000.0,0,NaN,NaN
3,395384,1070623204,1307194,Rappipay,2023-09-08,2024-07-22,2024-04-30,2024-07-31,2862200.0,1923813.0,1,GRADUADO_CREDITO,Tradicional
4,266503,3193745298,868155,Davivienda,2022-01-28,2022-08-17,NaT,2022-08-17,10134600.0,5000000.0,1,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
482092,394302,1110510087,1303299,Sistecredito,2023-09-04,NaT,NaT,NaT,418950.0,419950.0,0,NaN,NaN
482093,365958,3208396599,1201156,Codensa,2023-05-02,NaT,NaT,NaT,2732100.0,7887420.0,0,NaN,NaN
482094,2287,3208859986,4722,Falabella,2018-10-18,NaT,NaT,NaT,1398270.0,1000000.0,0,NaN,NaN
482095,483157,94412583,1628465,Banco de Bogota,2024-08-26,NaT,NaT,NaT,20175750.0,20175750.0,0,NaN,NaN


In [ ]:
def running_in_colab():
    try:
        import google.colab  # noqa
        return True
    except ImportError:
        return False

if running_in_colab():
    !pip install --quiet rapidfuzz

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 38.4 MB/s eta 0:00:00


In [ ]:
from rapidfuzz import process, fuzz
import pandas as pd
import re

nombres_correctos = [
    'Bancolombia', 'Banco Falabella', 'Banco de Bogotá', 'Banco Davivienda',
    'Scotiabank Colpatria', 'BBVA Colombia', 'SisteCredito', 'Banco AV Villas',
    'Banco de Occidente', 'Alkomprar', 'Tuya', 'Codensa', 'Itaú',
    'Serfinanza', 'Credivalores', 'Banco Popular', 'Rappipay',
    'Banco Finandina', 'Banco Caja Social', 'Covinoc', 'Bancoomeva',
    'Rapicredit', 'Credijamar', 'Flamingo', 'Zinobe', 'Bancamia',
    'Refinancia', 'Compensar', 'Pichincha', 'Colsubsidio', 'Mundo Mujer',
    'Agaval', 'Sistemcobro', 'Serlefin', 'Rappicard', 'Fincomercio',
    'GRUPO JURIDICO DEUDU', 'AECSA', 'Sufi', 'Comultrasan', 'Fundación',
    'Cobrando', 'Aslegal', 'Coltefinanciera', 'Reestructura', 'Nu',
    'Juancho te Presta', 'Muebles', 'JOHN', 'GNB Sudameris', 'Confiar',
    'Baninca', 'Systemgroup', 'Efecty'
]

# Bancos que se usan con el prefijo "banco" completo como referencia
bancos_excepciones = ['Banco de Bogotá', 'Bancolombia', 'Bancoomeva']

# ==============================
# 2. Sinónimos / patrones por banco (texto original, sin limpiar)
# ==============================
PATRONES_POR_BANCO = {
    'Bancolombia': [
        'bancolombia', 'contento bancolombia', 'qnt bancolombia', 'sufi'
    ],
    'Banco Davivienda': [
        'davivienda', 'intercredito davivienda', 'gestiones profesionales davivienda',
        'inversionistas estratégicos davivienda', 'inversionistas estrategicos davivienda',
        'management davivienda', 'davivienda cobrado sas', 'davivienda cobrando sas',
        'deudu davivienda', 'qnt davivienda'
    ],
    'BBVA Colombia': [
        'bbva', 'cobrando bbva', 'beta bbva', 'cobranzas beta origen: bbva',
        'aecsa bbva', 'grupo juridico bbva', 'grupo jurídico bbva', 'qnt bbva'
    ],
    'Banco Falabella': [
        'falabella', 'bfalabella', 'bancofalab', 'bancofalab citisumma',
        'banco falabella casa de cobro', 'eyc falabella',
        'cobrando falabella', 'acr logros f ori falabella',
        'deudu falabella', 'deudo falabella', 'citisumma falabella',
        'logros factoring falabella'
    ],
    'Tuya': [
        ' tuya', 'viva-tuya', 'viva tuya', 'qnt tuya', 'aecsa tuya',
        'tuya contacto soluciones', 'tuya s.a contactosol', 'tuya s a contactosol',
        'tuya contacto soluciones', 'qnt tuya',
        'éxito', 'exito', 'carulla', 'Alkosto', 'Corbeta'
    ],
    'Scotiabank Colpatria': [
        'scotiabank', 'skotiabank', 'colpatria', 'peruzzi skotiabank colpatria',
        'serlefin colpatria', 'adamantine scotiabank', 'gc andino colpatria',
        'scotiabank citibank', 'qnt colpatria', 'crc colpatria',
        'grupo consulto colpatria', 'grupo consultor andino colpatria',
        'gr.consulto colpatria', 'Codensa'
    ],
    'Banco de Bogotá': [
        'banco de bogota', 'banco de bogotá', 'qnt bogota', 'qnt bogotá',
        'crear país banco de bogotá', 'crear pais banco de bogota'
    ],
    'Banco de Occidente': [
        'banco de occidente', 'qnt banco de occidente', 'deudu-banco de occidente',
        'deudu banco de occidente'
    ],
    'Banco Popular': [
        'banco popular', 'banco popular casa de cobro', 'banco popular contactosol',
        'banco popular contactosolsas', 'banco popular citisumma',
        'banco popular-adcore', 'deudu banco popular', 'peruzzicol bcopopular'
    ],
    'Banco AV Villas': [
        'av villas', 'banco av villas', 'grupo consultor andino av villas',
        'grupo juridico av villas', 'grupo jurídico av villas',
        'ae csa av villas', 'aecsa av villas', 'crear país banco av villas',
        'crear pais banco av villas', 'deudu av villas', 'qnt av villas'
    ],
    'Banco Caja Social': [
        'banco caja social', 'caja social', 'pic caja social'
    ],
    'Bancoomeva': [
        'banco coomeva', 'bancoomeva', 'coomeva'
    ],
    'Bancamia': [
        'bancamia', 'bancamia s.a', 'bancamia s a'
    ],
    'Mundo Mujer': [
        'banco mundo mujer', 'fundacion de la mujer', 'fundación de la mujer'
    ],
    'SisteCredito': [
        'sistecredito', 'sistecrédito'
    ],
    'Covinoc': ['covinoc'],
    'Compensar': ['compensar'],
    'Pichincha': ['pichincha', 'pichincha educativo'],
    'Agaval': ['agaval'],
    'Banco Finandina': [
        'finandina', 'finandina incomercio', 'finandina incomercio'
    ],
    'Fincomercio': ['fincomercio'],
    'Serfinanza': [
        'serfinanza', 'serfinansa', 'serfinanza contactosol',
        'contacto solucion serfinanza', 'contacto solución serfinanza'
    ],
    'Credijamar': [
        'credijamar', 'muebles jamar'
    ],
    'Juancho te Presta': ['juancho te presta'],
    'Rapicredit': ['rapicredit'],
    'Zinobe': ['zinobe'],
    'Coltefinanciera': ['coltefinanciera'],
    'Sistemcobro': ['sistemcobro'],
    'Systemgroup': ['systemgroup'],
    'Baninca': ['baninca'],
    'GNB Sudameris': ['gnb sudameris'],
    'Confiar': ['confiar'],
    'AECSA': ['aecsa'],
    'Comultrasan': ['comultrasan'],
    'Nu': ['nu bank', 'nubank', 'logros factoring nubank'],
    'Rappi':['Rappicard', 'Rappipay'],
    'Lulo Bank': ['LuloBank', 'Lulo Banck'],
    'Banco union': ['QNT GIROS&FINANZAS']

}

# ==============================
# 3. Alias manuales sobre texto LIMPIO
# ==============================
alias_manuales = {
    'bfalabella': 'Banco Falabella',
    'contactosol': 'Banco Falabella',
    'qnt itau': 'Itaú',
    'itau helm': 'Itaú',
    'itau corpbanca': 'Itaú',
    'viva tuya': 'Tuya',
    'carulla': 'Tuya',
    'sufi': 'Bancolombia',
    'adamantine scotiabank': 'Scotiabank Colpatria',
    'gc andino colpatria': 'Scotiabank Colpatria',
    'beta bbva': 'BBVA Colombia',
    'banco popular': 'Banco Popular',
    'qnt bogota': 'Banco de Bogotá',
    'banco caja social': 'Banco Caja Social',
    'banco av villas': 'Banco AV Villas',
    'banco davivienda': 'Banco Davivienda',
    'bancofalab citisumma': 'Banco Falabella',
    'exito': 'Tuya',
    'éxito': 'Tuya',
    'Alkosto': 'Tuya',
    'Corbeta': 'Tuya',
    'Rappipay': 'Rappi',
    'Rappicard': 'Rappi',
    'Lulo Banck': 'Lulo Bank',
    'LuloBank': 'Lulo Bank',
    'QNT GIROS&FINANZAS': 'Banco Union',
    'Codensa': 'Scotiabank Colpatria'


}

# ==============================
# 4. Función de limpieza
# ==============================
def limpiar_texto(texto):
    """
    Limpia y normaliza el texto: minúsculas, elimina caracteres no alfabéticos
    (excepto acentos y ñ/ü), elimina palabras irrelevantes y quita espacios dobles.
    """
    texto = str(texto).lower()

    # Dejar solo letras, acentos, ñ, ü y espacios
    texto = re.sub(r'[^a-záéíóúñü\s]', ' ', texto)

    # Eliminar palabras "de relleno" frecuentes en reparadoras / BPO
    texto = re.sub(
        r'\b('
        r'grupo|juridico|jurídico|sas|sa|s a|ltda|suma|financiera|'
        r'contactosol|contacto|solucion|soluciones|citisumma|'
        r'cobrando|cobranzas|adcore|logros|factoring|origen|origem|'
        r'gestiones|gestion|profesionales|bpo|inversionistas|'
        r'estrategicos|estratégicos|casa|de|cobro|servicios|'
        r'creditos|credito|abogados|asociados|'
        r'outsourcing|risk|patrimonio|autonomo|autónomo|central|'
        r'inversiones|valora|punto|com|puntocom|activos|'
        r'recuperacion|recuperación|financiera|financiero|'
        r'asesores|asociados|gest|prof|eyc|gca|summa'
        r')\b',
        '',
        texto
    )

    # Espacios múltiples -> uno solo
    texto = re.sub(r'\s+', ' ', texto).strip()
    return texto

# ==============================
# 5. Claves de referencia para fuzzy
# ==============================
claves_referencia = {
    (n.lower() if n in bancos_excepciones else re.sub(r'^banco\s*', '', n.lower())): n
    for n in nombres_correctos
}

# ==============================
# 6. Mapeo por patrón (texto original)
# ==============================
def mapear_por_patron(nombre_incorrecto):
    """
    Intenta identificar el banco a partir del texto ORIGINAL (sin limpiar),
    buscando los patrones definidos en PATRONES_POR_BANCO.
    """
    if pd.isna(nombre_incorrecto):
        return None

    texto = str(nombre_incorrecto).lower()

    for banco_estandar, patrones in PATRONES_POR_BANCO.items():
        for patron in patrones:
            if patron in texto:
                return banco_estandar

    return None

# ==============================
# 7. Función principal de corrección
# ==============================
def corregir_nombre(nombre_incorrecto):
    """
    Corrige un nombre de banco usando 3 capas:
    1. Mapeo por patrón sobre el texto original.
    2. Limpieza de texto + alias manuales.
    3. Fuzzy matching contra claves_referencia.
    """
    # Si es NaN/None, lo dejamos igual
    if pd.isna(nombre_incorrecto):
        return nombre_incorrecto

    # 1. Intentar primero con patrones evidentes en el texto original
    banco_patron = mapear_por_patron(nombre_incorrecto)
    if banco_patron is not None:
        return banco_patron

    # 2. Limpiar texto
    limpio = limpiar_texto(nombre_incorrecto)

    # Si después de limpiar no queda nada, devolvemos el original
    if limpio == '':
        return nombre_incorrecto

    # 3. Alias manuales sobre texto limpio
    for alias, banco_estandar in alias_manuales.items():
        if alias in limpio:
            return banco_estandar

    # 4. Fuzzy matching con RapidFuzz
    mejor_match, score, _ = process.extractOne(
        limpio,
        claves_referencia.keys(),
        scorer=fuzz.token_set_ratio
    )

    # Si la similitud es alta, usamos el banco estándar; si no, dejamos el texto original
    return claves_referencia[mejor_match] if score > 70 else nombre_incorrecto

# ==============================
# 8. Aplicar al DataFrame
# ==============================

# Crear nueva columna estandarizada en reparadoras_df
reparadoras_df['BANCOS_ESTANDAR'] = reparadoras_df['BANCO'].apply(corregir_nombre)


In [ ]:
reparadoras_df['BANCOS_ESTANDAR'].unique()

array(['Banco Popular', 'Scotiabank Colpatria', 'Colsubsidio', 'Rappipay',
       'Banco Davivienda', 'Banco Falabella', 'Bancolombia',
       'SisteCredito', 'Flamingo', 'Banco de Bogotá', 'Codensa',
       'Refinancia', 'BBVA Colombia', 'Covinoc', 'Tuya', 'Alkosto',
       'Mundo Mujer', 'Alkomprar', 'Banco Caja Social',
       'Banco de Occidente', 'Banco AV Villas', 'Rapicredit',
       'Serfinanza', 'Credijamar', 'Itaú', 'Bancoomeva', 'Agaval',
       'Juriscoop', 'Zinobe', 'Compensar', 'Serlefin', 'Credivalores',
       'Pichincha', 'Sistemcobro', 'Banco Finandina', 'Aslegal',
       'Bancamia', 'Rappicard', 'Comultrasan', 'Juancho te Presta',
       'GRUPO JURIDICO DEUDU', None, 'GNB Sudameris', 'Fincomercio',
       'FGA', 'JOHN', 'Confiar', 'Lulo Banck', 'Cobrando',
       'Coltefinanciera', 'Nu', 'Reestructura', 'Citi summa', 'AECSA',
       'Contacto solución', 'Baninca', 'GCA', 'QNT SAS', 'UTRAHUILCA',
       'COOPCENTRAL', 'LuloBank', 'Garantias Comunitarias',
       'Inve

In [ ]:
# Crear la columna estandarizada
reparadoras_df['BANCOS_ESTANDAR'] = reparadoras_df['BANCO'].apply(corregir_nombre)

# Reordenar columnas para poner BANCOS_ESTANDAR justo después de BANCO
cols = list(reparadoras_df.columns)

# Sacamos las columnas en orden
cols.remove('BANCOS_ESTANDAR')
cols.insert(cols.index('BANCO') + 1, 'BANCOS_ESTANDAR')

# Reasignar al DataFrame
reparadoras_df = reparadoras_df[cols]

#creamos df_bank

In [ ]:
import pandas as pd

# Convertir a datetime si aún no lo están
reparadoras_df['FECHA DE INICIO'] = pd.to_datetime(reparadoras_df['FECHA DE INICIO'], errors='coerce')

# Fecha límite
fecha_corte = pd.Timestamp('2020-01-01')

# Filtrar filas donde ambas fechas sean >= 2020-01-01
reparadoras_df = reparadoras_df[
    (reparadoras_df['FECHA DE INICIO'] >= fecha_corte)
].copy()

In [ ]:
reparadoras_df

,BEREX ID,REFERENCIA,ID DEUDA,BANCO,BANCOS_ESTANDAR,FECHA DE INICIO,FECHA DE PAGO,FECHA DE BAJA,FECHA DE GRADUACIÓN,D_BRAVO,PAB,CREDITO,STATUS,TIPO DE PRICING
0,432204,3003925128,1442113,Banco Popular,Banco Popular,2024-02-14,2024-12-03,2025-03-31,NaT,4207850.0,1250000.0,0,BAJA_SOLICITADA,Pricing
1,193957,46622,645217,Scotiabank Colpatria,Scotiabank Colpatria,2021-01-12,2021-12-24,2021-12-31,NaT,1655500.0,800000.0,1,CREDITO_PARCIAL,Tradicional
2,293068,3014445964,950756,Colsubsidio,Colsubsidio,2022-06-06,2023-03-16,NaT,2023-03-16,498300.0,242000.0,0,NaN,NaN
3,395384,1070623204,1307194,Rappipay,Rappipay,2023-09-08,2024-07-22,2024-04-30,2024-07-31,2862200.0,1923813.0,1,GRADUADO_CREDITO,Tradicional
4,266503,3193745298,868155,Davivienda,Banco Davivienda,2022-01-28,2022-08-17,NaT,2022-08-17,10134600.0,5000000.0,1,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
482091,401706,3118174684,1330347,Éxito,Tuya,2023-10-02,NaT,NaT,NaT,2000250.0,2000250.0,0,NaN,NaN
482092,394302,1110510087,1303299,Sistecredito,SisteCredito,2023-09-04,NaT,NaT,NaT,418950.0,419950.0,0,NaN,NaN
482093,365958,3208396599,1201156,Codensa,Codensa,2023-05-02,NaT,NaT,NaT,2732100.0,7887420.0,0,NaN,NaN
482095,483157,94412583,1628465,Banco de Bogota,Banco de Bogotá,2024-08-26,NaT,NaT,NaT,20175750.0,20175750.0,0,NaN,NaN


In [ ]:
# 1. REFERENCIAS que se asocian a más de un BEREX ID
ref_conflicto = (
    reparadoras_df
    .groupby('REFERENCIA')['BEREX ID']
    .nunique()
    .reset_index()
)

# nos quedamos solo con las que tienen más de un BEREX ID
ref_conflicto = ref_conflicto[ref_conflicto['BEREX ID'] > 1]

print("REFERENCIAS con más de un BEREX ID distinto:")
print(ref_conflicto)

# Si quieres ver el detalle fila a fila:
detalle_ref_conflicto = reparadoras_df.merge(
    ref_conflicto[['REFERENCIA']],
    on='REFERENCIA',
    how='inner'
).sort_values(['REFERENCIA', 'BEREX ID'])

detalle_ref_conflicto.head()

REFERENCIAS con más de un BEREX ID distinto:
       REFERENCIA  BEREX ID
0               0         2
1039   1032428332         2
4804   3002037554         2
4846   3002071199         2
4867   3002088935         2
...           ...       ...
94167  3508766881         2
94483       40772         2
94729       41707         2
96476       45518         2
98065       48993         2

[785 rows x 2 columns]


,BEREX ID,REFERENCIA,ID DEUDA,BANCO,BANCOS_ESTANDAR,FECHA DE INICIO,FECHA DE PAGO,FECHA DE BAJA,FECHA DE GRADUACIÓN,D_BRAVO,PAB,CREDITO,STATUS,TIPO DE PRICING
3633,145526,0,497659,Banco Popular,Banco Popular,2020-05-12,NaT,2020-06-30,NaT,3001950.0,NaN,<NA>,NaN,NaN
3634,145526,0,497658,Falabella,Banco Falabella,2020-05-12,NaT,2020-06-30,NaT,2350700.0,NaN,<NA>,NaN,NaN
2310,159845,0,542040,Bancolombia,Bancolombia,2020-08-03,NaT,2020-08-31,NaT,2016000.0,NaN,<NA>,NaN,NaN
2311,159845,0,542039,Bancolombia,Bancolombia,2020-08-03,NaT,2020-08-31,NaT,1977150.0,NaN,<NA>,NaN,NaN
2312,159845,0,542038,Bancolombia,Bancolombia,2020-08-03,NaT,2020-08-31,NaT,1197000.0,NaN,<NA>,NaN,NaN


In [ ]:
detalle_ref_conflicto

,BEREX ID,REFERENCIA,ID DEUDA,BANCO,BANCOS_ESTANDAR,FECHA DE INICIO,FECHA DE PAGO,FECHA DE BAJA,FECHA DE GRADUACIÓN,D_BRAVO,PAB,CREDITO,STATUS,TIPO DE PRICING
3633,145526,0,497659,Banco Popular,Banco Popular,2020-05-12,NaT,2020-06-30,NaT,3001950.0,NaN,<NA>,NaN,NaN
3634,145526,0,497658,Falabella,Banco Falabella,2020-05-12,NaT,2020-06-30,NaT,2350700.0,NaN,<NA>,NaN,NaN
2310,159845,0,542040,Bancolombia,Bancolombia,2020-08-03,NaT,2020-08-31,NaT,2016000.0,NaN,<NA>,NaN,NaN
2311,159845,0,542039,Bancolombia,Bancolombia,2020-08-03,NaT,2020-08-31,NaT,1977150.0,NaN,<NA>,NaN,NaN
2312,159845,0,542038,Bancolombia,Bancolombia,2020-08-03,NaT,2020-08-31,NaT,1197000.0,NaN,<NA>,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4573,255532,45518,834845,Banco de Bogota,Banco de Bogotá,2021-11-25,NaT,2022-03-31,NaT,2106300.0,NaN,<NA>,NaN,NaN
3295,199543,48993,661979,Scotiabank Colpatria,Scotiabank Colpatria,2021-02-06,NaT,2021-03-31,NaT,1555817.0,NaN,<NA>,NaN,NaN
3296,199543,48993,661978,Éxito,Tuya,2021-02-06,NaT,2021-03-31,NaT,635800.0,NaN,<NA>,NaN,NaN
3297,199543,48993,661980,Scotiabank Colpatria,Scotiabank Colpatria,2021-02-06,NaT,2021-03-31,NaT,4670668.0,NaN,<NA>,NaN,NaN


In [ ]:
detalle_ref_conflicto['D_BRAVO'].sum()

np.float64(49478000700.21001)

In [ ]:
reparadoras_df.columns

Index(['BEREX ID', 'REFERENCIA', 'ID DEUDA', 'BANCO', 'BANCOS_ESTANDAR',
       'FECHA DE INICIO', 'FECHA DE PAGO', 'FECHA DE BAJA',
       'FECHA DE GRADUACIÓN', 'D_BRAVO', 'PAB', 'CREDITO', 'STATUS',
       'TIPO DE PRICING'],
      dtype='object')

In [ ]:
import pandas as pd

# Convertir FECHA DE INICIO a datetime
reparadoras_df['FECHA DE INICIO'] = pd.to_datetime(
    reparadoras_df['FECHA DE INICIO'],
    errors='coerce'
)

# Crear FECHA_ORIGEN como mes-año
reparadoras_df['FECHA_ORIGEN'] = (
    reparadoras_df['FECHA DE INICIO'].dt.to_period('M').astype(str)
)

# Crear df_bank con una fila por cada combinación única
df_bank = (
    reparadoras_df[['BANCOS_ESTANDAR', 'FECHA_ORIGEN']]
    .drop_duplicates()
    .sort_values(['BANCOS_ESTANDAR', 'FECHA_ORIGEN'])
    .reset_index(drop=True)
)

In [ ]:
# 1. Agrupar en reparadoras_df por banco y FECHA_ORIGEN
agg_df = (
    reparadoras_df
    .groupby(['BANCOS_ESTANDAR', 'FECHA_ORIGEN'])
    .agg(
        SUMA_D_BRAVO=('D_BRAVO', 'sum'),   # suma de D_BRAVO
        Q_DEUDAS=('D_BRAVO', 'size')       # número de filas (deudas)
    )
    .reset_index()
)

# 2. Unir estos agregados a df_bank
df_bank = df_bank.merge(
    agg_df,
    on=['BANCOS_ESTANDAR', 'FECHA_ORIGEN'],
    how='left'
)

In [ ]:
df_bank

,BANCOS_ESTANDAR,FECHA_ORIGEN,SUMA_D_BRAVO,Q_DEUDAS
0,ACYR-Activos y recuperación,2022-07,10467600.0,2.0
1,ACYR-Activos y recuperación,2022-09,8558000.0,1.0
2,ACYR-Activos y recuperación,2023-04,11122700.0,4.0
3,ACYR-Activos y recuperación,2023-12,3532200.0,2.0
4,ADCORE,2021-07,11207900.0,1.0
...,...,...,...,...
3152,NaN,2023-10,NaN,NaN
3153,NaN,2023-11,NaN,NaN
3154,NaN,2024-04,NaN,NaN
3155,NaN,2024-09,NaN,NaN


#df_inicios

In [ ]:
df_bajas.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 313332 entries, 0 to 313331
Data columns (total 9 columns):
 #   Column        Non-Null Count   Dtype         
---  ------        --------------   -----         
 0   BEREX ID      313332 non-null  object        
 1   ID DEUDA      313332 non-null  object        
 2   drop_date     313182 non-null  datetime64[ns]
 3   BANCO         313273 non-null  object        
 4   status        313331 non-null  object        
 5   inserted_at   313331 non-null  datetime64[ns]
 6   active_debts  313332 non-null  int64         
 7   D_BRAVO       313331 non-null  float64       
 8   REFERENCIA    313332 non-null  object        
dtypes: datetime64[ns](2), float64(1), int64(1), object(5)
memory usage: 21.5+ MB


In [ ]:
df_activas.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 76187 entries, 0 to 76186
Data columns (total 13 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   BEREX ID             76187 non-null  object        
 1   FECHA DE GRADUACIÓN  243 non-null    datetime64[ns]
 2   ID DEUDA             76187 non-null  object        
 3   FECHA DE BAJA        1051 non-null   datetime64[ns]
 4   BANCO                76187 non-null  object        
 5   status               76187 non-null  object        
 6   FECHA DE INICIO      76187 non-null  datetime64[ns]
 7   CREDITO              76187 non-null  int64         
 8   mora_liq_o_hoy       76187 non-null  int64         
 9   D_BRAVO              76187 non-null  float64       
 10  FECHA DE PAGO        22692 non-null  datetime64[ns]
 11  PAB                  75462 non-null  float64       
 12  REFERENCIA           76187 non-null  object        
dtypes: datetime64[ns](4), float64(2

In [ ]:
df_liquidado_2025.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 69508 entries, 0 to 69507
Data columns (total 13 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   BEREX ID         69508 non-null  object        
 1   graduation_date  26875 non-null  datetime64[ns]
 2   debt_id          69508 non-null  int64         
 3   max              69508 non-null  datetime64[ns]
 4   id               69508 non-null  int64         
 5   drop_date        17366 non-null  datetime64[ns]
 6   BANCO            69408 non-null  object        
 7   inserted_at      69508 non-null  datetime64[ns]
 8   CREDITO          69508 non-null  bool          
 9   D_BRAVO          69508 non-null  float64       
 10  FECHA DE PAGO    69508 non-null  datetime64[ns]
 11  PAB              69508 non-null  float64       
 12  REFERENCIA       69508 non-null  object        
dtypes: bool(1), datetime64[ns](5), float64(2), int64(2), object(3)
memory usage: 6.4+ MB


In [ ]:
# Tomar solo las columnas requeridas y renombrar inserted_at
df_inicio = df_bajas[['BEREX ID', 'REFERENCIA', 'inserted_at']].copy()

# Renombrar la columna inserted_at a FECHA_DE_INICIO
df_inicio.rename(columns={'inserted_at': 'FECHA_DE_INICIO'}, inplace=True)

In [ ]:
# Tomar solo las columnas requeridas y renombrar inserted_at
df_i = df_liquidado_2025[['BEREX ID', 'REFERENCIA', 'inserted_at']].copy()

# Renombrar la columna inserted_at a FECHA_DE_INICIO
df_i.rename(columns={'inserted_at': 'FECHA_DE_INICIO'}, inplace=True)

In [ ]:
df_inicio = pd.concat([df_inicio, df_activas, df_i], ignore_index=True)

In [ ]:
df_inicio.drop_duplicates(inplace=True)

In [ ]:
df_inicio

,BEREX ID,REFERENCIA,FECHA_DE_INICIO,FECHA DE GRADUACIÓN,ID DEUDA,FECHA DE BAJA,BANCO,status,FECHA DE INICIO,CREDITO,mora_liq_o_hoy,D_BRAVO,FECHA DE PAGO,PAB
0,506867,3024647576,2024-11-27,NaT,NaN,NaT,NaN,NaN,NaT,NaN,NaN,NaN,NaT,NaN
2,346434,3102856558,2023-02-13,NaT,NaN,NaT,NaN,NaN,NaT,NaN,NaN,NaN,NaT,NaN
5,535175,1020795948,2025-03-22,NaT,NaN,NaT,NaN,NaN,NaT,NaN,NaN,NaN,NaT,NaN
6,529640,3204752699,2025-02-28,NaT,NaN,NaT,NaN,NaN,NaT,NaN,NaN,NaN,NaT,NaN
10,434451,3204681956,2024-02-22,NaT,NaN,NaT,NaN,NaN,NaT,NaN,NaN,NaN,NaT,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
458934,412930,3113081139,2023-11-20,NaT,NaN,NaT,NaN,NaN,NaT,NaN,NaN,NaN,NaT,NaN
458952,309673,3174371010,2023-10-17,NaT,NaN,NaT,NaN,NaN,NaT,NaN,NaN,NaN,NaT,NaN
458961,399944,72429512,2023-09-27,NaT,NaN,NaT,NaN,NaN,NaT,NaN,NaN,NaN,NaT,NaN
458964,399197,3125562192,2023-09-25,NaT,NaN,NaT,NaN,NaN,NaT,NaN,NaN,NaN,NaT,NaN


In [ ]:
reparadoras_df

,BEREX ID,REFERENCIA,ID DEUDA,BANCO,BANCOS_ESTANDAR,FECHA DE INICIO,FECHA DE PAGO,FECHA DE BAJA,FECHA DE GRADUACIÓN,D_BRAVO,PAB,CREDITO,STATUS,TIPO DE PRICING,FECHA_ORIGEN
0,432204,3003925128,1442113,Banco Popular,Banco Popular,2024-02-14,2024-12-03,2025-03-31,NaT,4207850.0,1250000.0,0,BAJA_SOLICITADA,Pricing,2024-02
1,193957,46622,645217,Scotiabank Colpatria,Scotiabank Colpatria,2021-01-12,2021-12-24,2021-12-31,NaT,1655500.0,800000.0,1,CREDITO_PARCIAL,Tradicional,2021-01
2,293068,3014445964,950756,Colsubsidio,Colsubsidio,2022-06-06,2023-03-16,NaT,2023-03-16,498300.0,242000.0,0,NaN,NaN,2022-06
3,395384,1070623204,1307194,Rappipay,Rappipay,2023-09-08,2024-07-22,2024-04-30,2024-07-31,2862200.0,1923813.0,1,GRADUADO_CREDITO,Tradicional,2023-09
4,266503,3193745298,868155,Davivienda,Banco Davivienda,2022-01-28,2022-08-17,NaT,2022-08-17,10134600.0,5000000.0,1,NaN,NaN,2022-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
482091,401706,3118174684,1330347,Éxito,Tuya,2023-10-02,NaT,NaT,NaT,2000250.0,2000250.0,0,NaN,NaN,2023-10
482092,394302,1110510087,1303299,Sistecredito,SisteCredito,2023-09-04,NaT,NaT,NaT,418950.0,419950.0,0,NaN,NaN,2023-09
482093,365958,3208396599,1201156,Codensa,Codensa,2023-05-02,NaT,NaT,NaT,2732100.0,7887420.0,0,NaN,NaN,2023-05
482095,483157,94412583,1628465,Banco de Bogota,Banco de Bogotá,2024-08-26,NaT,NaT,NaT,20175750.0,20175750.0,0,NaN,NaN,2024-08


In [ ]:
import pandas as pd

# ----------------------------------------
# 1. Asegurar tipos de fecha
# ----------------------------------------
cols_fecha_reparadoras = ["FECHA DE INICIO", "FECHA DE PAGO", "FECHA DE BAJA", "FECHA DE GRADUACIÓN"]
for c in cols_fecha_reparadoras:
    reparadoras_df[c] = pd.to_datetime(reparadoras_df[c], errors="coerce")

df_inicio["FECHA_DE_INICIO"] = pd.to_datetime(df_inicio["FECHA_DE_INICIO"], errors="coerce")

# ----------------------------------------
# 2. Marcar filas donde hay fechas "imposibles"
#    - FECHA DE PAGO < FECHA DE INICIO  -> problema
#    - FECHA DE PAGO == FECHA DE INICIO -> también problema
#    - FECHA DE BAJA / GRADUACIÓN < FECHA DE INICIO -> problema
# ----------------------------------------
event_cols = ["FECHA DE PAGO", "FECHA DE BAJA", "FECHA DE GRADUACIÓN"]

mask_problema = pd.Series(False, index=reparadoras_df.index)

for col in event_cols:
    # Condición base: evento antes de la fecha de inicio
    cond = (
        reparadoras_df[col].notna()
        & (reparadoras_df[col] < reparadoras_df["FECHA DE INICIO"])
    )

    # Caso especial: si es FECHA DE PAGO, también marcar cuando es igual
    if col == "FECHA DE PAGO":
        cond |= (
            reparadoras_df[col].notna()
            & (reparadoras_df[col] == reparadoras_df["FECHA DE INICIO"])
        )

    mask_problema |= cond

# ----------------------------------------
# 3. Función para buscar nueva FECHA_DE_INICIO
#    usando df_inicio (primero por REFERENCIA, luego por BEREX ID)
# ----------------------------------------
def buscar_nueva_fecha(row):
    # Fechas de evento no nulas
    fechas_evento = [row[c] for c in event_cols if pd.notna(row[c])]
    if not fechas_evento:
        return pd.NaT

    # Límite máximo: la fecha de evento más temprana
    limite = min(fechas_evento)

    # Primero buscar por REFERENCIA
    cand = df_inicio[df_inicio["REFERENCIA"] == row["REFERENCIA"]]

    # Si no hay por REFERENCIA, buscar por BEREX ID
    if cand.empty:
        cand = df_inicio[df_inicio["BEREX ID"] == row["BEREX ID"]]

    if cand.empty:
        return pd.NaT

    # Solo fechas de inicio que sean <= límite (antes o igual al primer evento)
    cand_validas = cand[cand["FECHA_DE_INICIO"] <= limite]
    if cand_validas.empty:
        return pd.NaT

    # De las válidas, tomar la más reciente
    return cand_validas["FECHA_DE_INICIO"].max()

# Aplicar solo a filas problemáticas
reparadoras_df["NUEVA_FECHA_INICIO"] = pd.NaT
reparadoras_df.loc[mask_problema, "NUEVA_FECHA_INICIO"] = (
    reparadoras_df[mask_problema].apply(buscar_nueva_fecha, axis=1)
)

# ----------------------------------------
# 4. Actualizar FECHA DE INICIO y FECHA_ORIGEN
#    para las filas donde sí encontramos fecha nueva
# ----------------------------------------
mask_corregibles = mask_problema & reparadoras_df["NUEVA_FECHA_INICIO"].notna()

# Actualizar FECHA DE INICIO
reparadoras_df.loc[mask_corregibles, "FECHA DE INICIO"] = (
    reparadoras_df.loc[mask_corregibles, "NUEVA_FECHA_INICIO"]
)

# Actualizar FECHA_ORIGEN (año-mes de la nueva FECHA DE INICIO)
reparadoras_df.loc[mask_corregibles, "FECHA_ORIGEN"] = (
    reparadoras_df.loc[mask_corregibles, "FECHA DE INICIO"].dt.strftime("%Y-%m")
)

# ----------------------------------------
# 5. Borrar filas que tenían problema y NO se pudieron corregir
# ----------------------------------------
mask_sin_solucion = mask_problema & reparadoras_df["NUEVA_FECHA_INICIO"].isna()
reparadoras_df = reparadoras_df[~mask_sin_solucion].copy()

# Eliminar columna auxiliar
reparadoras_df.drop(columns=["NUEVA_FECHA_INICIO"], inplace=True)

# (Opcional) Resetear índice
reparadoras_df.reset_index(drop=True, inplace=True)

In [ ]:
reparadoras_df

,BEREX ID,REFERENCIA,ID DEUDA,BANCO,BANCOS_ESTANDAR,FECHA DE INICIO,FECHA DE PAGO,FECHA DE BAJA,FECHA DE GRADUACIÓN,D_BRAVO,PAB,CREDITO,STATUS,TIPO DE PRICING,FECHA_ORIGEN
0,432204,3003925128,1442113,Banco Popular,Banco Popular,2024-02-14,2024-12-03,2025-03-31,NaT,4207850.0,1250000.0,0,BAJA_SOLICITADA,Pricing,2024-02
1,193957,46622,645217,Scotiabank Colpatria,Scotiabank Colpatria,2021-01-12,2021-12-24,2021-12-31,NaT,1655500.0,800000.0,1,CREDITO_PARCIAL,Tradicional,2021-01
2,293068,3014445964,950756,Colsubsidio,Colsubsidio,2022-06-06,2023-03-16,NaT,2023-03-16,498300.0,242000.0,0,NaN,NaN,2022-06
3,395384,1070623204,1307194,Rappipay,Rappipay,2023-09-08,2024-07-22,2024-04-30,2024-07-31,2862200.0,1923813.0,1,GRADUADO_CREDITO,Tradicional,2023-09
4,266503,3193745298,868155,Davivienda,Banco Davivienda,2022-01-28,2022-08-17,NaT,2022-08-17,10134600.0,5000000.0,1,NaN,NaN,2022-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
414002,401706,3118174684,1330347,Éxito,Tuya,2023-10-02,NaT,NaT,NaT,2000250.0,2000250.0,0,NaN,NaN,2023-10
414003,394302,1110510087,1303299,Sistecredito,SisteCredito,2023-09-04,NaT,NaT,NaT,418950.0,419950.0,0,NaN,NaN,2023-09
414004,365958,3208396599,1201156,Codensa,Codensa,2023-05-02,NaT,NaT,NaT,2732100.0,7887420.0,0,NaN,NaN,2023-05
414005,483157,94412583,1628465,Banco de Bogota,Banco de Bogotá,2024-08-26,NaT,NaT,NaT,20175750.0,20175750.0,0,NaN,NaN,2024-08


In [ ]:
df_bajas

,BEREX ID,ID DEUDA,drop_date,BANCO,status,inserted_at,active_debts,D_BRAVO,REFERENCIA
0,506867,1716699.0,2025-05-20,Banco de Bogota,new,2024-11-27,2,7933200.0,3024647576
1,506867,1716698.0,2025-05-20,Banco de Bogota,new,2024-11-27,2,2618000.0,3024647576
2,346434,1131486.0,2025-04-29,Davivienda,negotiation,2023-02-13,3,53603000.0,3102856558
3,346434,1131485.0,2025-04-29,Bancolombia,negotiation,2023-02-13,3,35426600.0,3102856558
4,346434,1131484.0,2025-04-29,Éxito,liquidation_structured_payment,2023-02-13,3,15042500.0,3102856558
...,...,...,...,...,...,...,...,...,...
313327,219613,721433.0,2021-10-31,Davivienda,negotiation,2021-05-19,6,891399.0,76847
313328,219613,721432.0,2021-10-31,Davivienda,negotiation,2021-05-19,6,483940.0,76847
313329,219613,721434.0,2021-10-31,Banco de Bogota,negotiation,2021-05-19,6,5447488.0,76847
313330,219613,721437.0,2021-10-31,Scotiabank Colpatria,negotiation,2021-05-19,6,17845868.0,76847


#Timeline

In [ ]:
import pandas as pd

# =========================
# 1. Asegurar tipos de fecha
# =========================
for col in ['FECHA DE INICIO', 'FECHA DE PAGO', 'FECHA DE BAJA', 'FECHA DE GRADUACIÓN']:
    reparadoras_df[col] = pd.to_datetime(reparadoras_df[col], errors='coerce')

# ====================================================
# 1.1. Regla: si hay FECHA DE PAGO, no cuenta la BAJA
# ====================================================
mask_pago = reparadoras_df['FECHA DE PAGO'].notna()
reparadoras_df.loc[mask_pago, 'FECHA DE BAJA'] = pd.NaT

# ==========================================
# 2. Crear FECHA_ORIGEN (mes-año de inicio)
# ==========================================
reparadoras_df['FECHA_ORIGEN'] = reparadoras_df['FECHA DE INICIO'].dt.to_period('M').astype(str)

# =======================================================
# 3. Tabla de INICIO (D_BRAVO_INICIAL / Q_DEUDAS_INICIAL)
# =======================================================
inicio_df = (
    reparadoras_df
    .groupby(['BANCOS_ESTANDAR', 'FECHA_ORIGEN'])
    .agg(
        D_BRAVO_INICIAL=('D_BRAVO', 'sum'),
        Q_DEUDAS_INICIAL=('D_BRAVO', 'size')   # cuenta filas (deudas)
    )
    .reset_index()
)

inicio_df['MESES_DESPUES'] = 0
inicio_df['Suceso'] = 'Inicio'

# columnas de suceso como NaN para que encaje con la tabla de eventos
inicio_df['D_BRAVO_SUCESO'] = pd.NA
inicio_df['Q_DEUDAS_SUCESO'] = pd.NA

# ============================================================
# 4. Tabla de SUCESOS (Liquidación / Baja / Graduado por meses)
# ============================================================
# Pasar FECHA DE PAGO / FECHA DE BAJA / FECHA DE GRADUACIÓN a formato largo
eventos_long = reparadoras_df.melt(
    id_vars=['BANCOS_ESTANDAR', 'FECHA DE INICIO', 'FECHA_ORIGEN', 'D_BRAVO'],
    value_vars=['FECHA DE PAGO', 'FECHA DE BAJA', 'FECHA DE GRADUACIÓN'],
    var_name='tipo_fecha',
    value_name='FECHA_EVENTO'
)

# Nos quedamos solo con las filas donde sí hubo evento
eventos_long = eventos_long.dropna(subset=['FECHA_EVENTO'])

# Mapear el tipo de fecha a nombre de Suceso
map_suceso = {
    'FECHA DE PAGO': 'Liquidación',
    'FECHA DE BAJA': 'Baja',
    'FECHA DE GRADUACIÓN': 'Graduado'
}
eventos_long['Suceso'] = eventos_long['tipo_fecha'].map(map_suceso)

# Calcular MESES_DESPUES como diferencia en meses entre FECHA_EVENTO e inicio
eventos_long['MESES_DESPUES'] = (
    (eventos_long['FECHA_EVENTO'].dt.year - eventos_long['FECHA DE INICIO'].dt.year) * 12 +
    (eventos_long['FECHA_EVENTO'].dt.month - eventos_long['FECHA DE INICIO'].dt.month)
)

# Agregar por banco + cohorte + meses + suceso
sucesos_df = (
    eventos_long
    .groupby(['BANCOS_ESTANDAR', 'FECHA_ORIGEN', 'MESES_DESPUES', 'Suceso'])
    .agg(
        D_BRAVO_SUCESO=('D_BRAVO', 'sum'),
        Q_DEUDAS_SUCESO=('D_BRAVO', 'size')
    )
    .reset_index()
)

# columnas iniciales como NaN aquí
sucesos_df['D_BRAVO_INICIAL'] = pd.NA
sucesos_df['Q_DEUDAS_INICIAL'] = pd.NA

# =======================================
# 5. Unir INICIO + SUCESOS en un solo df
# =======================================
df_bank_timeline = pd.concat([inicio_df, sucesos_df], ignore_index=True)

df_bank_timeline = df_bank_timeline[
    [
        'BANCOS_ESTANDAR',
        'FECHA_ORIGEN',
        'MESES_DESPUES',
        'Suceso',
        'D_BRAVO_INICIAL',
        'Q_DEUDAS_INICIAL',
        'D_BRAVO_SUCESO',
        'Q_DEUDAS_SUCESO'
    ]
].sort_values(
    ['BANCOS_ESTANDAR', 'FECHA_ORIGEN', 'MESES_DESPUES', 'Suceso']
).reset_index(drop=True)

In [ ]:
df_bank_timeline

In [ ]:
# -*- coding: utf-8 -*-
import os
import json
import base64
from typing import Optional, Dict, Any

import pandas as pd
import numpy as np
import gspread
from google.oauth2.service_account import Credentials


# =========================================================
# 0) Helpers: entorno + MI_JSON (Colab o GitHub)
# =========================================================
def _running_in_colab() -> bool:
    try:
        import google.colab  # noqa: F401
        return True
    except Exception:
        return False


def _get_secret_from_colab(name: str) -> Optional[str]:
    try:
        from google.colab import userdata as _ud
        v = _ud.get(name)
        return v if v else None
    except Exception:
        return None


def _load_service_account_info(raw: str) -> Dict[str, Any]:
    """
    raw puede venir como:
      - JSON texto plano
      - Base64(JSON)
    """
    raw = (raw or "").strip()
    if not raw:
        raise RuntimeError("MI_JSON está vacío.")

    # Intento 1: JSON directo
    try:
        return json.loads(raw)
    except json.JSONDecodeError:
        pass

    # Intento 2: Base64(JSON)
    try:
        decoded = base64.b64decode(raw).decode("utf-8")
        return json.loads(decoded)
    except Exception as e:
        raise RuntimeError(
            "No pude interpretar MI_JSON. Debe ser JSON válido o Base64(JSON). "
            f"Error: {e}"
        )


def get_sa_info() -> Dict[str, Any]:
    raw = None
    if _running_in_colab():
        raw = _get_secret_from_colab("MI_JSON")

    if not raw:
        raw = os.environ.get("MI_JSON")

    if not raw:
        raise RuntimeError(
            "No se encontró MI_JSON.\n"
            "- En Colab: Entorno de ejecución -> Ver secretos -> MI_JSON\n"
            "- En GitHub Actions: agrega MI_JSON como secret y pásalo como env.\n"
            "  (puede ser JSON plano o Base64(JSON))"
        )

    return _load_service_account_info(raw)


# =========================================================
# 1) Construir df_bank_timeline desde reparadoras_df
# =========================================================
if "reparadoras_df" not in globals():
    raise RuntimeError("No existe el DataFrame 'reparadoras_df' en memoria.")

df = reparadoras_df.copy()

date_cols = ["FECHA DE INICIO", "FECHA DE PAGO", "FECHA DE BAJA", "FECHA DE GRADUACIÓN"]
missing = [c for c in date_cols + ["BANCOS_ESTANDAR", "D_BRAVO"] if c not in df.columns]
if missing:
    raise ValueError(f"Faltan columnas en reparadoras_df: {missing}")

# Asegurar tipos de fecha (sin mutar el DF original)
for col in date_cols:
    df[col] = pd.to_datetime(df[col], errors="coerce")

# FECHA_ORIGEN = cohorte mes-año de FECHA DE INICIO
df["FECHA_ORIGEN"] = df["FECHA DE INICIO"].dt.to_period("M").astype(str)

# ---- 1.1 Tabla de INICIO (D_BRAVO_INICIAL / Q_DEUDAS_INICIAL) ----
inicio_df = (
    df.groupby(["BANCOS_ESTANDAR", "FECHA_ORIGEN"], dropna=False)
      .agg(
          D_BRAVO_INICIAL=("D_BRAVO", "sum"),
          Q_DEUDAS_INICIAL=("D_BRAVO", "size"),
      )
      .reset_index()
)

inicio_df["MESES_DESPUES"] = 0
inicio_df["Suceso"] = "Inicio"

# columnas de suceso como NA para que encaje con la tabla de eventos
inicio_df["D_BRAVO_SUCESO"] = pd.NA
inicio_df["Q_DEUDAS_SUCESO"] = pd.NA

# ---- 1.2 Tabla de SUCESOS (Liquidación / Baja / Graduado por meses) ----
eventos_long = df.melt(
    id_vars=["BANCOS_ESTANDAR", "FECHA DE INICIO", "FECHA_ORIGEN", "D_BRAVO"],
    value_vars=["FECHA DE PAGO", "FECHA DE BAJA", "FECHA DE GRADUACIÓN"],
    var_name="tipo_fecha",
    value_name="FECHA_EVENTO",
)

# Solo filas con evento válido
eventos_long = eventos_long.dropna(subset=["FECHA_EVENTO", "FECHA DE INICIO"])

map_suceso = {
    "FECHA DE PAGO": "Liquidación",
    "FECHA DE BAJA": "Baja",
    "FECHA DE GRADUACIÓN": "Graduado",
}
eventos_long["Suceso"] = eventos_long["tipo_fecha"].map(map_suceso)

# MESES_DESPUES = diferencia en meses entre FECHA_EVENTO y FECHA DE INICIO
eventos_long["MESES_DESPUES"] = (
    (eventos_long["FECHA_EVENTO"].dt.year - eventos_long["FECHA DE INICIO"].dt.year) * 12
    + (eventos_long["FECHA_EVENTO"].dt.month - eventos_long["FECHA DE INICIO"].dt.month)
)

# (Opcional) si no quieres meses negativos (evento antes del inicio), descomenta:
# eventos_long = eventos_long[eventos_long["MESES_DESPUES"] >= 0]

sucesos_df = (
    eventos_long
    .groupby(["BANCOS_ESTANDAR", "FECHA_ORIGEN", "MESES_DESPUES", "Suceso"], dropna=False)
    .agg(
        D_BRAVO_SUCESO=("D_BRAVO", "sum"),
        Q_DEUDAS_SUCESO=("D_BRAVO", "size"),
    )
    .reset_index()
)

# columnas iniciales como NA aquí
sucesos_df["D_BRAVO_INICIAL"] = pd.NA
sucesos_df["Q_DEUDAS_INICIAL"] = pd.NA

# ---- 1.3 Unir INICIO + SUCESOS ----
df_bank_timeline = pd.concat([inicio_df, sucesos_df], ignore_index=True)

df_bank_timeline = (
    df_bank_timeline[
        [
            "BANCOS_ESTANDAR",
            "FECHA_ORIGEN",
            "MESES_DESPUES",
            "Suceso",
            "D_BRAVO_INICIAL",
            "Q_DEUDAS_INICIAL",
            "D_BRAVO_SUCESO",
            "Q_DEUDAS_SUCESO",
        ]
    ]
    .sort_values(["BANCOS_ESTANDAR", "FECHA_ORIGEN", "MESES_DESPUES", "Suceso"])
    .reset_index(drop=True)
)

print("✅ df_bank_timeline construido con forma:", df_bank_timeline.shape)


# =========================================================
# 2) Subir df_bank_timeline a Google Sheets (hoja 'Timeline')
# =========================================================
SPREADSHEET_URL = "https://docs.google.com/spreadsheets/d/1-Z9A8FIlVR0QtU7-A_HLtdhAMAOe0tFLDTcj_D0QlVM"
SHEET_NAME = "Timeline"

sa_info = get_sa_info()

creds = Credentials.from_service_account_info(
    sa_info,
    scopes=[
        "https://www.googleapis.com/auth/spreadsheets",
        "https://www.googleapis.com/auth/drive",
    ],
)

gc = gspread.authorize(creds)
sh = gc.open_by_url(SPREADSHEET_URL)

# Crear hoja si no existe
sheet_names = [ws.title for ws in sh.worksheets()]
if SHEET_NAME not in sheet_names:
    sh.add_worksheet(title=SHEET_NAME, rows="1000", cols="30")
    print(f"✔️ Hoja '{SHEET_NAME}' creada")
else:
    print(f"✔️ Hoja '{SHEET_NAME}' ya existe; será reemplazada")

ws = sh.worksheet(SHEET_NAME)

# Limpiar contenido anterior
ws.clear()

# Preparar DF para subida (Sheets no acepta NaN/inf en JSON)
df_to_upload = df_bank_timeline.copy()
df_to_upload = df_to_upload.replace([np.inf, -np.inf], np.nan).fillna("")

values = [df_to_upload.columns.tolist()] + df_to_upload.values.tolist()

# Subir (RAW para que no intente “interpretar”)
ws.update(values, value_input_option="RAW")

print(f"🚀 Datos subidos exitosamente a la hoja '{SHEET_NAME}'.")
print("Service Account:", sa_info.get("client_email"))

In [ ]:
# -*- coding: utf-8 -*-
import io
import os
import json
import base64
import time
import ssl
import socket
from typing import Optional, Dict, Any

import pandas as pd

from google.oauth2.service_account import Credentials
from googleapiclient.discovery import build
from googleapiclient.http import MediaIoBaseUpload
from googleapiclient.errors import HttpError


# =========================================================
# 0) Timeouts generales (útil para redes inestables)
# =========================================================
socket.setdefaulttimeout(300)


# =========================================================
# 1) Cargar MI_JSON (Colab o GitHub)
# =========================================================
def _running_in_colab() -> bool:
    try:
        import google.colab  # noqa: F401
        return True
    except Exception:
        return False


def _get_secret_from_colab(name: str) -> Optional[str]:
    try:
        # Ojo: según tu preferencia, MI_JSON siempre desde userdata en Colab
        from google.colab import userdata as _ud
        v = _ud.get(name)
        return v if v else None
    except Exception:
        return None


def _load_service_account_info(raw: str) -> Dict[str, Any]:
    """
    raw puede venir como:
      - JSON texto plano
      - Base64(JSON)
    """
    raw = (raw or "").strip()
    if not raw:
        raise RuntimeError("MI_JSON está vacío.")

    # Intento 1: JSON directo
    try:
        return json.loads(raw)
    except json.JSONDecodeError:
        pass

    # Intento 2: Base64(JSON)
    try:
        decoded = base64.b64decode(raw).decode("utf-8")
        return json.loads(decoded)
    except Exception as e:
        raise RuntimeError(
            "No pude interpretar MI_JSON. Debe ser JSON válido o Base64(JSON). "
            f"Error: {e}"
        )


def get_sa_info() -> Dict[str, Any]:
    raw = None

    # Colab
    if _running_in_colab():
        raw = _get_secret_from_colab("MI_JSON")

    # GitHub / local
    if not raw:
        raw = os.environ.get("MI_JSON")

    if not raw:
        raise RuntimeError(
            "No se encontró MI_JSON.\n"
            "- En Colab: Entorno de ejecución -> Ver secretos -> MI_JSON\n"
            "- En GitHub Actions: agrega MI_JSON como secret y pásalo como env.\n"
            "  (puede ser JSON plano o Base64(JSON))"
        )

    return _load_service_account_info(raw)


# =========================================================
# 2) Construir cliente Drive
# =========================================================
SCOPES = ["https://www.googleapis.com/auth/drive"]

sa_info = get_sa_info()
creds = Credentials.from_service_account_info(sa_info, scopes=SCOPES)
drive = build("drive", "v3", credentials=creds, cache_discovery=False)


# =========================================================
# 3) ID del archivo de destino (Drive)
# =========================================================
FILE_ID = "1L850Ymbi74sEC3vetvzxLmf0qr9HPf5Z"  # <- tu ID
print(f"📄 Actualizando archivo con ID: {FILE_ID}")


# =========================================================
# 4) Verificar DataFrame
# =========================================================
if "reparadoras_df" not in globals():
    raise RuntimeError("No existe el DataFrame 'reparadoras_df' en memoria.")

df_to_save = reparadoras_df.copy()


# =========================================================
# 5) Generar Excel en memoria
# =========================================================
buf = io.BytesIO()
with pd.ExcelWriter(buf, engine="openpyxl") as writer:
    df_to_save.to_excel(writer, index=False, sheet_name="C_liq")
buf.seek(0)


# =========================================================
# 6) Subir como nueva revisión (resumable upload)
# =========================================================
media = MediaIoBaseUpload(
    buf,
    mimetype="application/vnd.openxmlformats-officedocument.spreadsheetml.sheet",
    chunksize=2 * 1024 * 1024,  # 2 MB
    resumable=True,
)

request = drive.files().update(
    fileId=FILE_ID,
    media_body=media,
    fields="id, name, webViewLink",
    supportsAllDrives=True,
)

response = None
attempt = 0
max_attempts = 6

print("⏫ Iniciando subida reanudable por chunks...")

while response is None:
    try:
        status, response = request.next_chunk(num_retries=5)
        if status:
            print(f"Progreso: {int(status.progress() * 100)}%")

    except (HttpError, ssl.SSLEOFError, ConnectionError, OSError, socket.timeout) as e:
        attempt += 1
        if attempt >= max_attempts:
            raise RuntimeError(f"Falló la subida tras {attempt} intentos. Último error: {e}")

        sleep_s = 2 ** attempt
        print(f"⚠️ Error de red/subida. Reintentando en {sleep_s}s… ({attempt}/{max_attempts})")
        time.sleep(sleep_s)

print("✅ Subida exitosa (nueva revisión).")
print("📝 Nombre:", response.get("name"))
print("🔗 Enlace:", response.get("webViewLink"))
print("Service Account:", sa_info.get("client_email"))

In [ ]:
# -*- coding: utf-8 -*-
import os
import io
import json
import base64
import time
import socket
from typing import Any, Dict, Optional, List

import pandas as pd

from google.oauth2.service_account import Credentials
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError


# =========================================================
# 0) Ajustes generales
# =========================================================
# Timeout global (útil para hojas muy grandes / conexiones lentas)
socket.setdefaulttimeout(300)

SCOPES = [
    "https://www.googleapis.com/auth/drive",
    "https://www.googleapis.com/auth/spreadsheets",
]


# =========================================================
# 1) Helpers: detectar entorno y leer MI_JSON
# =========================================================
def _running_in_colab() -> bool:
    try:
        import google.colab  # noqa: F401
        return True
    except Exception:
        return False


def _get_secret_from_colab(name: str) -> Optional[str]:
    try:
        from google.colab import userdata as _ud
        v = _ud.get(name)
        return v if v else None
    except Exception:
        return None


def _load_service_account_info_from_raw(raw_json: str) -> Dict[str, Any]:
    """
    raw_json puede venir como:
      - JSON texto plano
      - Base64(JSON)
    """
    raw_json = raw_json.strip()
    # Intento 1: JSON directo
    try:
        return json.loads(raw_json)
    except json.JSONDecodeError:
        pass

    # Intento 2: Base64 -> JSON
    try:
        decoded = base64.b64decode(raw_json).decode("utf-8")
        return json.loads(decoded)
    except Exception as e:
        raise RuntimeError(
            "No pude interpretar MI_JSON. Asegúrate de que sea JSON válido "
            "o Base64(JSON). Error: " + str(e)
        )


def get_sa_info() -> Dict[str, Any]:
    """
    Prioridad:
      1) Colab: secreto MI_JSON (userdata)
      2) GitHub/Local: variable de entorno MI_JSON
         (puede ser JSON plano o Base64(JSON))
    """
    raw = None

    if _running_in_colab():
        raw = _get_secret_from_colab("MI_JSON")

    if not raw:
        raw = os.environ.get("MI_JSON")

    if not raw:
        raise RuntimeError(
            "No se encontró MI_JSON.\n"
            "- En Colab: Entorno de ejecución -> Ver secretos -> MI_JSON\n"
            "- En GitHub Actions: agrega MI_JSON como secret y expórtalo como env\n"
            "  (puede ser JSON plano o Base64(JSON))."
        )

    return _load_service_account_info_from_raw(raw)


# =========================================================
# 2) Helpers: Sheets (crear/obtener/limpiar/redimensionar)
# =========================================================
def get_sheets_client(sa_info: Dict[str, Any]):
    creds = Credentials.from_service_account_info(sa_info, scopes=SCOPES)
    sheets = build("sheets", "v4", credentials=creds, cache_discovery=False)
    return sheets, sa_info


def get_sheet_id(sheets, spreadsheet_id: str, title: str) -> Optional[int]:
    try:
        spreadsheet = sheets.spreadsheets().get(
            spreadsheetId=spreadsheet_id,
            fields="sheets(properties(sheetId,title,gridProperties(rowCount,columnCount)))"
        ).execute()
    except HttpError as e:
        raise RuntimeError(f"No se pudo leer el spreadsheet. Revisa ID/permisos. Detalle: {e}")

    for sh in spreadsheet.get("sheets", []):
        props = sh.get("properties", {})
        if props.get("title") == title:
            return int(props.get("sheetId"))
    return None


def add_sheet(sheets, spreadsheet_id: str, title: str, row_count: int, col_count: int) -> int:
    body = {
        "requests": [
            {
                "addSheet": {
                    "properties": {
                        "title": title,
                        "gridProperties": {
                            "rowCount": row_count,
                            "columnCount": col_count
                        }
                    }
                }
            }
        ]
    }
    resp = sheets.spreadsheets().batchUpdate(
        spreadsheetId=spreadsheet_id,
        body=body
    ).execute()

    return int(resp["replies"][0]["addSheet"]["properties"]["sheetId"])


def resize_sheet(sheets, spreadsheet_id: str, sheet_id: int, row_count: int, col_count: int):
    body = {
        "requests": [
            {
                "updateSheetProperties": {
                    "properties": {
                        "sheetId": sheet_id,
                        "gridProperties": {
                            "rowCount": row_count,
                            "columnCount": col_count
                        }
                    },
                    "fields": "gridProperties(rowCount,columnCount)"
                }
            }
        ]
    }
    sheets.spreadsheets().batchUpdate(
        spreadsheetId=spreadsheet_id,
        body=body
    ).execute()


def clear_sheet_values(sheets, spreadsheet_id: str, sheet_title: str):
    # Limpia todo el contenido visible de la pestaña (valores)
    sheets.spreadsheets().values().clear(
        spreadsheetId=spreadsheet_id,
        range=f"{sheet_title}!A:Z"
    ).execute()


# =========================================================
# 3) Helpers: preparar DF para Google Sheets
# =========================================================
def df_to_values(df: pd.DataFrame) -> List[List[Any]]:
    """
    Convierte DataFrame a matriz de valores para Google Sheets
    - datetime -> string
    - NaN / NA -> ""
    - blindado para columnas Int64 / boolean / string
    """
    df2 = df.copy()

    # datetime -> string
    for c in df2.columns:
        if pd.api.types.is_datetime64_any_dtype(df2[c]):
            df2[c] = df2[c].dt.strftime("%Y-%m-%d %H:%M:%S")

    # 🔑 CLAVE: convertir TODO a object antes de meter ""
    df2 = df2.astype("object")

    # NaN / NA -> ""
    df2 = df2.where(pd.notna(df2), "")

    return [df2.columns.tolist()] + df2.values.tolist()

# =========================================================
# 4) Escritura por chunks con reintentos
# =========================================================
def write_values_in_chunks(
    sheets,
    spreadsheet_id: str,
    sheet_title: str,
    values: List[List[Any]],
    chunk_size: int = 5000,
    max_retries: int = 5,
):
    """
    values incluye header en values[0].
    Escribe header en A1 y luego data desde A2 en chunks.
    """
    header = values[0]
    data = values[1:]
    total_rows = len(data)

    # 1) Header
    sheets.spreadsheets().values().update(
        spreadsheetId=spreadsheet_id,
        range=f"{sheet_title}!A1",
        valueInputOption="RAW",
        body={"values": [header]},
    ).execute()
    print("✅ Encabezados escritos en fila 1.")

    if total_rows == 0:
        print("ℹ️ No hay filas de datos para escribir.")
        return

    print(f"🧩 Enviando {total_rows} filas en chunks de {chunk_size}...")

    for start in range(0, total_rows, chunk_size):
        end = min(start + chunk_size, total_rows)
        chunk = data[start:end]

        # +2 porque fila 1 es header
        start_row = start + 2
        end_row = start_row + len(chunk) - 1
        range_ = f"{sheet_title}!A{start_row}"

        print(f"  → Escribiendo filas {start_row} a {end_row}...")

        for attempt in range(1, max_retries + 1):
            try:
                sheets.spreadsheets().values().update(
                    spreadsheetId=spreadsheet_id,
                    range=range_,
                    valueInputOption="RAW",
                    body={"values": chunk},
                ).execute()
                print(f"     ✅ Chunk {start_row}-{end_row} escrito (intento {attempt}).")
                break

            except HttpError as e:
                status = getattr(e.resp, "status", None)
                if status in (500, 502, 503, 504):
                    print(f"     ⚠️ HttpError {status} (server). Reintento {attempt}/{max_retries}...")
                    time.sleep(2 ** attempt)
                    if attempt == max_retries:
                        raise
                else:
                    print(f"     ❌ HttpError no recuperable ({status}): {e}")
                    raise

            except (TimeoutError, socket.timeout) as e:
                print(f"     ⏱️ Timeout ({type(e).__name__}). Reintento {attempt}/{max_retries}...")
                time.sleep(2 ** attempt)
                if attempt == max_retries:
                    raise

            except Exception as e:
                print(f"     ❌ Error inesperado: {e}")
                raise

    print("🎉 DataFrame completo escrito en la hoja por chunks.")


# =========================================================
# 5) CONFIG: tu Spreadsheet y nombre de hoja
# =========================================================
SPREADSHEET_ID = "1-Z9A8FIlVR0QtU7-A_HLtdhAMAOe0tFLDTcj_D0QlVM"
NEW_SHEET_NAME = "C_liq_reparadoras"

# Si quieres que siempre sobreescriba (borre valores antes de escribir):
CLEAR_BEFORE_WRITE = True

# Chunk recomendado (baja a 2000 si tu red es inestable)
CHUNK_SIZE = 5000


# =========================================================
# 6) MAIN
# =========================================================
def main():
    # 6.1) Validar DF en memoria
    if "reparadoras_df" not in globals():
        raise RuntimeError("No existe el DataFrame 'reparadoras_df' en memoria.")

    df_to_save = reparadoras_df.copy()
    rows, cols = df_to_save.shape
    total_cells_df = (rows + 1) * cols
    print(f"📏 reparadoras_df: {rows} filas x {cols} columnas = {total_cells_df} celdas (incluye header).")

    # 6.2) Auth
    sa_info = get_sa_info()
    sheets, sa_info = get_sheets_client(sa_info)
    print("✅ Cliente Sheets creado.")
    print("Service Account:", sa_info.get("client_email"))

    print(f"📄 Spreadsheet ID: {SPREADSHEET_ID}")
    print(f"🧾 Sheet destino: {NEW_SHEET_NAME}")

    # 6.3) Crear o encontrar hoja
    sheet_id = get_sheet_id(sheets, SPREADSHEET_ID, NEW_SHEET_NAME)

    if sheet_id is None:
        sheet_id = add_sheet(
            sheets,
            SPREADSHEET_ID,
            NEW_SHEET_NAME,
            row_count=max(rows + 1, 2),  # mínimo 2 filas
            col_count=max(cols, 1),      # mínimo 1 col
        )
        print(f"🆕 Hoja '{NEW_SHEET_NAME}' creada (sheetId={sheet_id}).")
    else:
        print(f"ℹ️ La hoja '{NEW_SHEET_NAME}' ya existe (sheetId={sheet_id}).")
        # Redimensionar a lo justo (opcional)
        resize_sheet(
            sheets,
            SPREADSHEET_ID,
            sheet_id,
            row_count=max(rows + 1, 2),
            col_count=max(cols, 1),
        )
        print(f"📐 Hoja '{NEW_SHEET_NAME}' redimensionada a {max(rows+1,2)} filas x {max(cols,1)} columnas.")

    # 6.4) Limpiar (opcional)
    if CLEAR_BEFORE_WRITE:
        try:
            clear_sheet_values(sheets, SPREADSHEET_ID, NEW_SHEET_NAME)
            print("🧹 Hoja limpiada (valores) antes de escribir.")
        except Exception as e:
            print(f"⚠️ No pude limpiar la hoja (continuo igual). Detalle: {e}")

    # 6.5) Preparar valores y escribir
    values = df_to_values(df_to_save)
    write_values_in_chunks(
        sheets=sheets,
        spreadsheet_id=SPREADSHEET_ID,
        sheet_title=NEW_SHEET_NAME,
        values=values,
        chunk_size=CHUNK_SIZE,
        max_retries=5,
    )

    print(f"🔗 Enlace: https://docs.google.com/spreadsheets/d/{SPREADSHEET_ID}/edit")


# Ejecutar
main()